# "저공해차 보급촉진을 위한 제도운영 지원 및 개선방안 마련" 데이터 분석

## library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

### data load

In [235]:
total_data = pd.DataFrame()
# 2020년
count = 0
for m in range(1, 13):
    y = 20
    s = f'raw/{y}년/20{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2021년
for m in range(1, 13):
    y = 21
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2022년
for m in range(1, 9):
    y = 22
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)

In [236]:
print(f'개별 데이터 개수 합산 : {count}')
print(f'총 데이터 개수 : {total_data.shape[0]}')

개별 데이터 개수 합산 : 11665822
총 데이터 개수 : 11665822


In [78]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


##### 20-22년 병합 데이터 출력

In [80]:
total_data.to_csv('2020-22년_급속충전.csv', encoding='utf-8')

##### 20-22년 병합 데이터 로드(다시 시작)

In [638]:
# total_data = pd.read_csv('2020-22년_급속충전.csv', index_col=0)
# total_data.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_17248\1573190630.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv('2020-22년_급속충전.csv')


In [639]:
df = total_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   충전소명        object 
 2   충전기ID       int64  
 3   지역          object 
 4   시군구         object 
 5   주소          object 
 6   충전기용량(kW)   float64
 7   충전기타입       object 
 8   충전시작일시      int64  
 9   충전종료일시      int64  
 10  충전시간        object 
 11  충전량         float64
 12  충전용량        object 
dtypes: float64(2), int64(4), object(7)
memory usage: 1.1+ GB


In [640]:
df = df.drop(['Unnamed: 0'], axis=1)

## 전처리
- 충전소명 : O
- 충전기ID : int64
- 지역 : O
- 시군구 : O
- 주소 : O
- 충전기용량(kW) : float64
- 충전기타입 : object 
- 충전시작일시 : int64  
- 충전종료일시 : int64  
- 충전시간 : object 
- 충전량 : float64
- 충전용량 : object 

### 특이사항
- 2020년 12월까지 충전기용량(kW)가 2021년 1월부터 충전용량으로 변경
    - 2020년 12월까지 데이터의 충전기용량(kW) 값을 충전용량으로 이동
- 충전용량 표기방법 변경
    - 2022년 3월까지 충전용량이 "100"으로 , 4월부터 "급속(100kW멀티)"으로 입력

In [641]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


In [642]:
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN


In [645]:
df[df['충전용량'].notnull() == True].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,11.95,50.0
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,4.86,50.0
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,3.29,100.0
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,7.52,50.0
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,21.66,100.0


### 충전시작시간 정리

In [646]:
# 충전시작일시의 데이터는 년~초 까지 모두 존재
temp = df['충전시작일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

0


In [647]:
ch_start = df['충전시작일시'].copy()
df['충전시작_년도'] = ch_start.astype(str).str[:4].astype(int)
df['충전시작_월'] = df['충전시작일시'].astype(str).str[4:6].astype(int)
df['충전시작_일'] = df['충전시작일시'].astype(str).str[6:8].astype(int)
df['충전시작_시'] = df['충전시작일시'].astype(str).str[8:10].astype(int)
df['충전시작_분'] = df['충전시작일시'].astype(str).str[10:12].astype(int)
df['충전시작_초'] = df['충전시작일시'].astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17


In [648]:
df['충전시작일시(변환)'] = df['충전시작_년도'].astype(str) + '-' +  df['충전시작_월'].astype(str) + '-' + df['충전시작_일'].astype(str) + ' ' + df['충전시작_시'].astype(str) + ':' + df['충전시작_분'].astype(str) + ':' + df['충전시작_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-1-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-1-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-1-5 0:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-1-20 12:2:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-1-23 8:54:17


In [649]:
df['충전시작일시(변환)'] = pd.to_datetime(df['충전시작일시(변환)'])
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


### 충전종료시간 정리

In [650]:
df[df['충전종료일시'] == ' ']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)


In [651]:
# 충전종료일시의 데이터는 년~일까지만 있는 데이터가 71311개 존재
    # 해당 데이터 뒤에 시분초를 '000000'으로 추가하는 작업 필요함.
temp = df['충전종료일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

71311


In [652]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,31.09,급속(50kW),2022,8,31,23,59,18,2022-08-31 23:59:18
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,46.51,급속(200kW동시),2022,8,31,23,59,20,2022-08-31 23:59:20
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,21.97,급속(50kW),2022,8,31,23,59,31,2022-08-31 23:59:31
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,27.60,급속(50kW),2022,8,31,23,59,34,2022-08-31 23:59:34
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,9.77,급속(100kW멀티),2022,8,31,23,59,46,2022-08-31 23:59:46


In [653]:
temp = df['충전종료일시'].to_list()

In [654]:
for i, one in enumerate(temp):
    if len(str(one)) == 8:
        print(i)
        break

1578


In [655]:
temp[1578]

20200118

In [656]:
temp2 = [str(i) for i in temp]
temp2[1578]

'20200118'

In [657]:
for i, one in enumerate(temp2):
    if len(one) == 8:
        temp = one + '000000'
        temp2[i] = temp

In [658]:
temp2[1578]

'20200118000000'

In [659]:
df['충전종료일시'] = temp2
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


In [660]:
df.iloc[1578, :]

충전소명                동해웰빙레포츠타운 주차장
충전기ID                          21
지역                            강원도
시군구                           동해시
주소                 강원도 동해시 덕골길 10
충전기용량(kW)                   200.0
충전기타입                        DC콤보
충전시작일시             20200118162253
충전종료일시             20200118000000
충전시간                     00:06:49
충전량                          1.91
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                          1
충전시작_일                         18
충전시작_시                         16
충전시작_분                         22
충전시작_초                         53
충전시작일시(변환)    2020-01-18 16:22:53
Name: 1578, dtype: object

In [661]:
ch_time = df['충전종료일시'].copy()
df['충전종료_년도'] = ch_time.astype(str).str[:4].astype(int)
df['충전종료_월'] = ch_time.astype(str).str[4:6].astype(int)
df['충전종료_일'] = ch_time.astype(str).str[6:8].astype(int)
df['충전종료_시'] = ch_time.astype(str).str[8:10].astype(int)
df['충전종료_분'] = ch_time.astype(str).str[10:12].astype(int)
df['충전종료_초'] = ch_time.astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,19,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,14,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,0,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,12,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,8,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54


In [662]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,...,23,59,18,2022-08-31 23:59:18,2022,9,1,0,40,34
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,...,23,59,20,2022-08-31 23:59:20,2022,9,1,0,19,4
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,...,23,59,31,2022-08-31 23:59:31,2022,9,1,0,42,51
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,...,23,59,34,2022-08-31 23:59:34,2022,9,1,0,40,57
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,...,23,59,46,2022-08-31 23:59:46,2022,9,1,0,37,0


#### 충전종료일시-초 오류 데이터
- 1495개

In [663]:
df[df['충전종료_초'] >= 60].shape

(1495, 25)

In [664]:
over_s_idx = df[df['충전종료_초'] >= 60].index
len(over_s_idx)

1495

In [665]:
for i in over_s_idx:
    df.loc[i, '충전종료_분'] += 1
    df.loc[i, '충전종료_초'] -= 60

In [666]:
df.iloc[1960560, :]

충전소명                  포천시의회 야외주차장
충전기ID                          44
지역                            경기도
시군구                           포천시
주소                 경기도 포천시 중앙로 87
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201023102909
충전종료일시             20201023103099
충전시간                     00:03:10
충전량                          2.36
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         23
충전시작_시                         10
충전시작_분                         29
충전시작_초                          9
충전시작일시(변환)    2020-10-23 10:29:09
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         23
충전종료_시                         10
충전종료_분                         31
충전종료_초                         39
Name: 1960560, dtype: object

#### 충전종료일시-분 오류 데이터
- 3270개

In [667]:
df[df['충전종료_분'] >= 60].shape

(3270, 25)

In [668]:
over_m_idx = df[df['충전종료_분'] >= 60].index
len(over_m_idx)

3270

In [669]:
for i in over_m_idx:
    df.loc[i, '충전종료_시'] += 1
    df.loc[i, '충전종료_분'] -= 60

In [670]:
df.iloc[1964993, :]

충전소명                  포천시의회 야외주차장
충전기ID                          45
지역                            경기도
시군구                           포천시
주소                 경기도 포천시 중앙로 87
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201023174301
충전종료일시             20201023176497
충전시간                     00:36:36
충전량                         25.44
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         23
충전시작_시                         17
충전시작_분                         43
충전시작_초                          1
충전시작일시(변환)    2020-10-23 17:43:01
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         23
충전종료_시                         18
충전종료_분                          5
충전종료_초                         37
Name: 1964993, dtype: object

#### 충전종료일시-시 오류 데이터
- 71개

In [671]:
df[df['충전종료_시'] >= 24].shape

(71, 25)

In [672]:
over_h_idx = df[df['충전종료_시'] >= 24].index
len(over_h_idx)

71

In [673]:
for i in over_h_idx:
    df.loc[i, '충전종료_일'] += 1
    df.loc[i, '충전종료_시'] -= 24

In [674]:
df.iloc[2168170, :]

충전소명             제천종합운동장 대형버스 주차장
충전기ID                          41
지역                           충청북도
시군구                           제천시
주소               충청북도 제천시 숭의로 101
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201030234823
충전종료일시             20201030237223
충전시간                     00:40:00
충전량                         23.22
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         30
충전시작_시                         23
충전시작_분                         48
충전시작_초                         23
충전시작일시(변환)    2020-10-30 23:48:23
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         31
충전종료_시                          0
충전종료_분                         12
충전종료_초                         23
Name: 2168170, dtype: object

#### 충전종료일시-일 오류 데이터
- 3개(index) : 3232880, 2440150, 2775050

In [675]:
df[(df['충전종료_월'] == 1) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [676]:
df[(df['충전종료_년도'] == 2021) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([3232880], dtype='int64')

In [677]:
df[(df['충전종료_년도'] == 2022) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([], dtype='int64')

In [678]:
# 2020년은 윤년
df[(df['충전종료_년도'] == 2020) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 29)].index

Int64Index([], dtype='int64')

In [679]:
df[(df['충전종료_월'] == 3) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [680]:
df[(df['충전종료_월'] == 4) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [681]:
df[(df['충전종료_월'] == 5) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [682]:
df[(df['충전종료_월'] == 6) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [683]:
df[(df['충전종료_월'] == 7) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [684]:
df[(df['충전종료_월'] == 8) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [685]:
df[(df['충전종료_월'] == 9) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [686]:
df[(df['충전종료_월'] == 10) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [687]:
df[(df['충전종료_월'] == 11) & (df['충전종료_일'] > 30)].index

Int64Index([2440150], dtype='int64')

In [688]:
df[(df['충전종료_월'] == 12) & (df['충전종료_일'] > 31)].index

Int64Index([2775050], dtype='int64')

In [689]:
check_col = ['충전종료_년도', '충전종료_월', '충전종료_일']
df.loc[3232880, check_col]

충전종료_년도    2021
충전종료_월        2
충전종료_일       29
Name: 3232880, dtype: object

In [690]:
df.loc[3232880, '충전종료_월'] = 3
df.loc[3232880, '충전종료_일'] = 1
df.loc[3232880, check_col]

충전종료_년도    2021
충전종료_월        3
충전종료_일        1
Name: 3232880, dtype: object

In [691]:
df.loc[2440150, check_col]

충전종료_년도    2020
충전종료_월       11
충전종료_일       31
Name: 2440150, dtype: object

In [692]:
df.loc[2440150, '충전종료_월'] = 12
df.loc[2440150, '충전종료_일'] = 1
df.loc[2440150, check_col]

충전종료_년도    2020
충전종료_월       12
충전종료_일        1
Name: 2440150, dtype: object

In [693]:
df.loc[2775050, check_col]

충전종료_년도    2020
충전종료_월       12
충전종료_일       32
Name: 2775050, dtype: object

In [694]:
df.loc[2775050, '충전종료_년도'] = 2021
df.loc[2775050, '충전종료_월'] = 1
df.loc[2775050, '충전종료_일'] = 1
df.loc[2775050, check_col]

충전종료_년도    2021
충전종료_월        1
충전종료_일        1
Name: 2775050, dtype: object

#### 충전종료시간(변환)

In [695]:
df['충전종료일시(변환)'] = df['충전종료_년도'].astype(str) + '-' +  df['충전종료_월'].astype(str) + '-' + df['충전종료_일'].astype(str) + ' ' + df['충전종료_시'].astype(str) + ':' + df['충전종료_분'].astype(str) + ':' + df['충전종료_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-1-17 20:14:17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-1-30 15:6:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-1-5 1:13:29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-1-20 12:47:8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-1-23 9:25:54


In [696]:
df['충전종료일시(변환)'] = pd.to_datetime(df['충전종료일시(변환)'])

In [697]:
df['충전시간(계산)'] = df['충전종료일시(변환)'] - df['충전시작일시(변환)']
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


### 충전기용량과 충전용량 합치기

In [698]:
df[df['충전시작_년도'] == 2020].tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796609,탄천제2호 공영주차장,1,서울특별시,강남구,"서울특별시 강남구 일원동 4-49 , 주차장 입구 오른쪽",50.0,DC차데모+AC3상+DC콤보,20201219120138,20201219124144,00:40:00,...,38,2020-12-19 12:01:38,2020,12,19,12,41,44,2020-12-19 12:41:44,0 days 00:40:06
2796610,온천장역 공영주차장,1,부산광역시,동래구,부산광역시 동래구 중앙대로 1495-10 공영주차장,50.0,DC차데모+AC3상+DC콤보,20201219120210,20201219124217,00:40:00,...,10,2020-12-19 12:02:10,2020,12,19,12,42,17,2020-12-19 12:42:17,0 days 00:40:07
2796611,화서휴게소(상주방향),1,경상북도,상주시,경상북도 상주시 화서면 터골길 51-25,50.0,DC차데모+AC3상+DC콤보,20201219120513,20201219124254,00:37:28,...,13,2020-12-19 12:05:13,2020,12,19,12,42,54,2020-12-19 12:42:54,0 days 00:37:41
2796612,삼양동주민센터,1,제주특별자치도,제주시,"제주특별자치도 제주시 지석13길 6 , 주차장",50.0,DC차데모+AC3상+DC콤보,20201219121419,20201219124313,00:28:47,...,19,2020-12-19 12:14:19,2020,12,19,12,43,13,2020-12-19 12:43:13,0 days 00:28:54
2796613,거창휴게소(옥포방향),1,경상남도,거창군,경상남도 거창군 가조면 도리,50.0,DC차데모+AC3상+DC콤보,20201219120725,20201219124405,00:36:34,...,25,2020-12-19 12:07:25,2020,12,19,12,44,5,2020-12-19 12:44:05,0 days 00:36:40


In [699]:
df[df['충전시작_년도'] == 2021].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,...,50,2021-01-01 09:22:50,2021,1,1,9,58,16,2021-01-01 09:58:16,0 days 00:35:26
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,...,26,2021-01-01 09:51:26,2021,1,1,9,58,18,2021-01-01 09:58:18,0 days 00:06:52
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,...,52,2021-01-01 09:47:52,2021,1,1,9,58,19,2021-01-01 09:58:19,0 days 00:10:27
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,...,23,2021-01-01 09:13:23,2021,1,1,9,58,19,2021-01-01 09:58:19,0 days 00:44:56
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,...,59,2021-01-01 09:15:59,2021,1,1,9,58,35,2021-01-01 09:58:35,0 days 00:42:36


In [701]:
temp = df[df['충전시작_년도'] == 2020]['충전기용량(kW)'].to_list()
temp[-5:]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [702]:
len(temp)

2796614

In [703]:
temp2 = df[df['충전시작_년도'] == 2021]['충전용량'].to_list()
temp2[-5:]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [704]:
len(temp2)

4259845

In [705]:
temp3 = df[df['충전시작_년도'] == 2022]['충전용량'].to_list()
temp3[-5:]

['급속(50kW)', '급속(200kW동시)', '급속(50kW)', '급속(50kW)', '급속(100kW멀티)']

In [706]:
len(temp3)

4609363

In [707]:
len(temp + temp2 + temp3)

11665822

In [708]:
df['충전용량'] = temp + temp2 + temp3
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


In [709]:
df.iloc[:5, :12]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,50.0
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,50.0
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,50.0
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,100.0
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,50.0


##### 일시 분리 데이터 출력

In [710]:
df.to_csv('2020-22년_급속충전_일시분리.csv', encoding='utf-8')

##### 일시 분리 데이터 로드(다시 시작)

In [5]:
# df = pd.read_csv('2020-22년_급속충전_일시분리.csv', index_col=0)
# df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_7780\927583139.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2020-22년_급속충전_일시분리.csv')


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   충전소명        object 
 2   충전기ID       int64  
 3   지역          object 
 4   시군구         object 
 5   주소          object 
 6   충전기용량(kW)   float64
 7   충전기타입       object 
 8   충전시작일시      int64  
 9   충전종료일시      int64  
 10  충전시간        object 
 11  충전량         float64
 12  충전용량        object 
 13  충전시작_년도     int64  
 14  충전시작_월      int64  
 15  충전시작_일      int64  
 16  충전시작_시      int64  
 17  충전시작_분      int64  
 18  충전시작_초      int64  
 19  충전시작일시(변환)  object 
 20  충전종료_년도     int64  
 21  충전종료_월      int64  
 22  충전종료_일      int64  
 23  충전종료_시      int64  
 24  충전종료_분      int64  
 25  충전종료_초      int64  
 26  충전종료일시(변환)  object 
 27  충전시간(계산)    object 
dtypes: float64(2), int64(16), object(10)
memory usage: 2.4+ GB


In [7]:
df = df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype  
---  ------      -----  
 0   충전소명        object 
 1   충전기ID       int64  
 2   지역          object 
 3   시군구         object 
 4   주소          object 
 5   충전기용량(kW)   float64
 6   충전기타입       object 
 7   충전시작일시      int64  
 8   충전종료일시      int64  
 9   충전시간        object 
 10  충전량         float64
 11  충전용량        object 
 12  충전시작_년도     int64  
 13  충전시작_월      int64  
 14  충전시작_일      int64  
 15  충전시작_시      int64  
 16  충전시작_분      int64  
 17  충전시작_초      int64  
 18  충전시작일시(변환)  object 
 19  충전종료_년도     int64  
 20  충전종료_월      int64  
 21  충전종료_일      int64  
 22  충전종료_시      int64  
 23  충전종료_분      int64  
 24  충전종료_초      int64  
 25  충전종료일시(변환)  object 
 26  충전시간(계산)    object 
dtypes: float64(2), int64(15), object(10)
memory usage: 2.3+ GB


### 필요없는 열 제거

In [8]:
# mod_df = df.drop(['충전기용량(kW)', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', 
#          '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초'], axis=1)
# mod_df.head()

### raw 충전시간과 계산값 비교

In [9]:
df['충전시간'].head()

0    00:40:00
1    00:30:31
2    00:15:35
3    00:41:00
4    00:31:14
Name: 충전시간, dtype: object

In [10]:
df['충전시간'].dtype

dtype('O')

#### 충전시간 오류 값 처리
- 7개 제거

In [11]:
# temp = pd.to_timedelta(df['충전시간']) # only leading negative signs are allowed 에러 발생
temp = pd.to_timedelta(df['충전시간'], errors='coerce')

In [12]:
temp.isnull().sum()

7

In [13]:
temp = temp.to_frame()

In [14]:
temp[temp['충전시간'].isnull() == True]

,충전시간
1207828,NaT
1352351,NaT
1362905,NaT
1372346,NaT
6796417,NaT
7111195,NaT
11392158,NaT


In [15]:
null_idx = temp[temp['충전시간'].isnull() == True].index
for i in null_idx:
    print(df.loc[i, '충전시간'], df.loc[i, '충전시작일시(변환)'])

00:-1:-4 2020-07-06 09:05:47
00:-4:-1 2020-07-05 15:39:18
00:-1:-2 2020-07-06 09:05:47
-6:-2:-5 2020-07-01 19:35:59
00:-4:-2 2021-12-30 17:18:27
00:-1:-3 2022-01-14 17:53:48
-1:-4:-4 2022-08-19 03:00:11


In [16]:
df.shape

(11665822, 27)

In [17]:
df2 = df.drop(null_idx, axis=0)
df2.shape

(11665815, 27)

In [18]:
temp = pd.to_timedelta(df2['충전시간'])
temp.head()

0   0 days 00:40:00
1   0 days 00:30:31
2   0 days 00:15:35
3   0 days 00:41:00
4   0 days 00:31:14
Name: 충전시간, dtype: timedelta64[ns]

In [19]:
df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
df2['충전시간(계산)'] = pd.to_timedelta(df2['충전시간(계산)'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11665815 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int64          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기용량(kW)   float64        
 6   충전기타입       object         
 7   충전시작일시      int64          
 8   충전종료일시      int64          
 9   충전시간        timedelta64[ns]
 10  충전량         float64        
 11  충전용량        object         
 12  충전시작_년도     int64          
 13  충전시작_월      int64          
 14  충전시작_일      int64          
 15  충전시작_시      int64          
 16  충전시작_분      int64          
 17  충전시작_초      int64          
 18  충전시작일시(변환)  object         
 19  충전종료_년도     int64          
 20  충전종료_월      int64          
 21  충전종료_일      int64          
 22  충전종료_시      int64          
 23  충전종료_분      int64          
 24  충전종료_초      int64     

In [20]:
df2['충전시간'].describe()

count                     11665815
mean     0 days 00:34:21.381945453
std      0 days 01:20:05.913477577
min                0 days 00:00:00
25%                0 days 00:20:55
50%                0 days 00:34:07
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [21]:
df2['충전시간(계산)'].describe()

count                     11665815
mean     0 days 00:29:18.506520633
std      4 days 08:44:42.009406092
min          -14905 days +09:10:08
25%                0 days 00:21:28
50%                0 days 00:34:49
75%                0 days 00:40:24
max               14 days 08:40:25
Name: 충전시간(계산), dtype: object

In [22]:
df2[df2['충전시간(계산)'] == '-14905 days +09:10:08']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2120856,단양관광안내소(단양팔경),1,충청북도,단양군,충청북도 단양군 단양읍 고수리 149-1,50.0,DC차데모+AC3상+DC콤보,20201030144953,19800110000001,3 days 10:47:44,...,53,2020-10-30 14:49:53,1980,1,10,0,0,1,1980-01-10 00:00:01,-14905 days +09:10:08


#### 충전시간 데이터 확인

In [23]:
df2[df2['충전시간'] == '0 days 00:00:00'].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
31211,(재)문화엑스포,11,경상북도,경주시,경상북도 경주시 경감로 614,200.0,DC콤보,20200109145420,20200109145420,0 days,...,20,2020-01-09 14:54:20,2020,1,9,14,54,20,2020-01-09 14:54:20,0 days 00:00:00
61525,종합사회복지관,12,충청남도,공주시,충청남도 공주시 의당면 의당로 257,200.0,DC콤보,20200114192124,20200114192132,0 days,...,24,2020-01-14 19:21:24,2020,1,14,19,21,32,2020-01-14 19:21:32,0 days 00:00:08
96952,해남군청 주차장,12,전라남도,해남군,전라남도 해남군 해남읍 수성리 183-5,200.0,DC콤보,20200124160600,20200124160601,0 days,...,0,2020-01-24 16:06:00,2020,1,24,16,6,1,2020-01-24 16:06:01,0 days 00:00:01
118208,롯데 빅마켓,3,서울특별시,영등포구,"서울특별시 영등포구 영중로 125, 지상주차장(외부 1F)",100.0,DC콤보,20200122012647,20200122012716,0 days,...,47,2020-01-22 01:26:47,2020,1,22,1,27,16,2020-01-22 01:27:16,0 days 00:00:29
174295,이마트 파주운정점,11,경기도,파주시,경기도 파주시 한울로 123,200.0,DC콤보,20200107165228,20200107165255,0 days,...,28,2020-01-07 16:52:28,2020,1,7,16,52,55,2020-01-07 16:52:55,0 days 00:00:27


In [24]:
df2[df2['충전시간'] == '0 days 00:00:00'].shape

(5702, 27)

In [25]:
df2['충전량'].dtype

dtype('float64')

In [26]:
df2['충전량'].head()

0    25.47
1    17.57
2     6.98
3    15.30
4    21.92
Name: 충전량, dtype: float64

In [27]:
df2[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0) ][['충전시간', '충전량']]

,충전시간,충전량
31211,0 days,1.18
61525,0 days,14.58
96952,0 days,13.33
118208,0 days,15.79
174295,0 days,22.51
...,...,...
11571988,0 days,18.70
11572694,0 days,23.41
11607868,0 days,7.44
11618047,0 days,4.50


In [28]:
df2[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0) ].shape

(5702, 27)

#### 충전시간(계산) 음수 값 제거
- 52366개

In [29]:
n_ch_time_idx = df2[df2['충전시간(계산)'] < '0'].index
len(n_ch_time_idx)

52366

In [30]:
df2 = df2.drop(n_ch_time_idx)

In [31]:
df2['충전시간'].describe()

count                     11613449
mean     0 days 00:33:51.200344617
std      0 days 01:19:04.781739278
min                0 days 00:00:00
25%                0 days 00:20:53
50%                0 days 00:34:02
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [32]:
df2['충전시간(계산)'].describe()

count                     11613449
mean     0 days 00:34:58.600275335
std      0 days 01:23:19.668917229
min                0 days 00:00:00
25%                0 days 00:21:39
50%                0 days 00:34:56
75%                0 days 00:40:25
max               14 days 08:40:25
Name: 충전시간(계산), dtype: object

In [33]:
df2['충전시간(계산)'].astype(str)[:5]

0    0 days 00:40:46
1    0 days 00:31:00
2    0 days 00:16:01
3    0 days 00:44:53
4    0 days 00:31:37
Name: 충전시간(계산), dtype: object

In [34]:
df2['충전시간'][:5]

0   0 days 00:40:00
1   0 days 00:30:31
2   0 days 00:15:35
3   0 days 00:41:00
4   0 days 00:31:14
Name: 충전시간, dtype: timedelta64[ns]

In [35]:
df2['충전시간'].dt.seconds[:5]

0    2400
1    1831
2     935
3    2460
4    1874
Name: 충전시간, dtype: int64

In [36]:
temp = df2['충전시간(계산)'] - df2['충전시간']
temp.describe()

count                     11613449
mean     0 days 00:01:07.399930718
std      0 days 01:01:06.908441070
min              -1 days +00:00:23
25%                0 days 00:00:00
50%                0 days 00:00:13
75%                0 days 00:01:42
max               14 days 08:32:12
dtype: object

In [37]:
temp[temp < '0'].shape

(817340,)

In [38]:
df2['raw값과_계산값_차이'] = df2['충전시간(계산)'] - df2['충전시간']

In [39]:
df2[df2['raw값과_계산값_차이'] > '14 days']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이
8510493,포항실내사격장,11,경상북도,포항시,경상북도 포항시 북구 용흥동 4-9,NaN,DC콤보,20220305094551,20220319182616,0 days 00:08:13,...,2022-03-05 09:45:51,2022,3,19,18,26,16,2022-03-19 18:26:16,14 days 08:40:25,14 days 08:32:12


In [40]:
# df2 = df2.drop(['raw값과_계산값_차이'], axis=1)

In [41]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int64          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기용량(kW)     float64        
 6   충전기타입         object         
 7   충전시작일시        int64          
 8   충전종료일시        int64          
 9   충전시간          timedelta64[ns]
 10  충전량           float64        
 11  충전용량          object         
 12  충전시작_년도       int64          
 13  충전시작_월        int64          
 14  충전시작_일        int64          
 15  충전시작_시        int64          
 16  충전시작_분        int64          
 17  충전시작_초        int64          
 18  충전시작일시(변환)    object         
 19  충전종료_년도       int64          
 20  충전종료_월        int64          
 21  충전종료_일        int64          
 22  충전종료_시        int64          
 23  충전종료_

In [44]:
df2.iloc[:, :14].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 14 columns):
 #   Column     Dtype          
---  ------     -----          
 0   충전소명       object         
 1   충전기ID      int64          
 2   지역         object         
 3   시군구        object         
 4   주소         object         
 5   충전기용량(kW)  float64        
 6   충전기타입      object         
 7   충전시작일시     int64          
 8   충전종료일시     int64          
 9   충전시간       timedelta64[ns]
 10  충전량        float64        
 11  충전용량       object         
 12  충전시작_년도    int64          
 13  충전시작_월     int64          
dtypes: float64(2), int64(5), object(6), timedelta64[ns](1)
memory usage: 1.3+ GB


In [45]:
df2.iloc[:, 14:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 14 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전시작_일        int64          
 1   충전시작_시        int64          
 2   충전시작_분        int64          
 3   충전시작_초        int64          
 4   충전시작일시(변환)    object         
 5   충전종료_년도       int64          
 6   충전종료_월        int64          
 7   충전종료_일        int64          
 8   충전종료_시        int64          
 9   충전종료_분        int64          
 10  충전종료_초        int64          
 11  충전종료일시(변환)    object         
 12  충전시간(계산)      timedelta64[ns]
 13  raw값과_계산값_차이  timedelta64[ns]
dtypes: int64(10), object(2), timedelta64[ns](2)
memory usage: 1.3+ GB


### 데이터 절약

In [47]:
pd.options.display.float_format = '{:.5f}'.format

In [49]:
time_cols = ['충전시작_년도', '충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']
df2[time_cols].describe()

,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
count,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000
mean,2021.16059,6.20425,15.80842,13.72640,29.47740,29.49963,2021.16062,6.20430,15.80841,14.01221,29.54277,29.45744
std,0.77937,3.33015,8.80106,4.95829,17.33414,17.31996,0.77937,3.33017,8.80110,5.07576,17.34527,17.33907
min,2020.00000,1.00000,1.00000,0.00000,0.00000,0.00000,2020.00000,1.00000,1.00000,0.00000,0.00000,0.00000
25%,2021.00000,3.00000,8.00000,10.00000,14.00000,14.00000,2021.00000,3.00000,8.00000,11.00000,15.00000,14.00000
50%,2021.00000,6.00000,16.00000,14.00000,29.00000,29.00000,2021.00000,6.00000,16.00000,14.00000,30.00000,29.00000
75%,2022.00000,9.00000,23.00000,17.00000,45.00000,45.00000,2022.00000,9.00000,23.00000,18.00000,45.00000,44.00000
max,2022.00000,12.00000,31.00000,23.00000,59.00000,59.00000,2022.00000,12.00000,31.00000,23.00000,59.00000,59.00000


In [53]:
df2[['충전시작_년도', '충전종료_년도']] = df2[['충전시작_년도', '충전종료_년도']].astype('int16')

In [54]:
df2[['충전시작_년도', '충전종료_년도']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   충전시작_년도  int16
 1   충전종료_년도  int16
dtypes: int16(2)
memory usage: 132.9 MB


In [55]:
df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 10 columns):
 #   Column  Dtype
---  ------  -----
 0   충전시작_월  int64
 1   충전시작_일  int64
 2   충전시작_시  int64
 3   충전시작_분  int64
 4   충전시작_초  int64
 5   충전종료_월  int64
 6   충전종료_일  int64
 7   충전종료_시  int64
 8   충전종료_분  int64
 9   충전종료_초  int64
dtypes: int64(10)
memory usage: 974.6 MB


In [57]:
df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']] = df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']].astype('int8')
df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 10 columns):
 #   Column  Dtype
---  ------  -----
 0   충전시작_월  int8 
 1   충전시작_일  int8 
 2   충전시작_시  int8 
 3   충전시작_분  int8 
 4   충전시작_초  int8 
 5   충전종료_월  int8 
 6   충전종료_일  int8 
 7   충전종료_시  int8 
 8   충전종료_분  int8 
 9   충전종료_초  int8 
dtypes: int8(10)
memory usage: 199.4 MB


In [59]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int64          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기용량(kW)     float64        
 6   충전기타입         object         
 7   충전시작일시        int64          
 8   충전종료일시        int64          
 9   충전시간          timedelta64[ns]
 10  충전량           float64        
 11  충전용량          object         
 12  충전시작_년도       int16          
 13  충전시작_월        int8           
 14  충전시작_일        int8           
 15  충전시작_시        int8           
 16  충전시작_분        int8           
 17  충전시작_초        int8           
 18  충전시작일시(변환)    object         
 19  충전종료_년도       int16          
 20  충전종료_월        int8           
 21  충전종료_일        int8           
 22  충전종료_시        int8           
 23  충전종료_

In [61]:
df2['충전기ID'].describe()

count   11613449.00000
mean           8.59347
std           15.08407
min            1.00000
25%            1.00000
50%            1.00000
75%           12.00000
max           98.00000
Name: 충전기ID, dtype: float64

In [62]:
df2['충전기ID'].info()

<class 'pandas.core.series.Series'>
Int64Index: 11613449 entries, 0 to 11665821
Series name: 충전기ID
Non-Null Count     Dtype
--------------     -----
11613449 non-null  int64
dtypes: int64(1)
memory usage: 177.2 MB


In [63]:
df2['충전기ID'] = df2['충전기ID'].astype('int16')
df2['충전기ID'].info()

<class 'pandas.core.series.Series'>
Int64Index: 11613449 entries, 0 to 11665821
Series name: 충전기ID
Non-Null Count     Dtype
--------------     -----
11613449 non-null  int16
dtypes: int16(1)
memory usage: 110.8 MB


In [64]:
df2['충전시간'].describe()

count                     11613449
mean     0 days 00:33:51.200344617
std      0 days 01:19:04.781739278
min                0 days 00:00:00
25%                0 days 00:20:53
50%                0 days 00:34:02
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [65]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기용량(kW)     float64        
 6   충전기타입         object         
 7   충전시작일시        int64          
 8   충전종료일시        int64          
 9   충전시간          timedelta64[ns]
 10  충전량           float64        
 11  충전용량          object         
 12  충전시작_년도       int16          
 13  충전시작_월        int8           
 14  충전시작_일        int8           
 15  충전시작_시        int8           
 16  충전시작_분        int8           
 17  충전시작_초        int8           
 18  충전시작일시(변환)    object         
 19  충전종료_년도       int16          
 20  충전종료_월        int8           
 21  충전종료_일        int8           
 22  충전종료_시        int8           
 23  충전종료_

In [73]:
df2 = df2.drop(['충전기용량(kW)'], axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

#### 충전시간 정리 파일로 출력

In [90]:
df2.to_csv('2020-22년_급속충전-충전시간정리.csv', encoding='utf-8')

In [93]:
# df2.to_excel('2020-22년_급속충전-충전시간정리.xlsx', encoding='utf-8') # error : 파일 너무 커서 저장 실패

##### 충전시간 정리 파일 로드(다시 시작)

In [24]:
df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0)
df2.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_15028\3254198657.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
dtypes: float64(1), int64(15), object(11)
memory usage: 2.4+ GB


### 지역별 충전시간, 충전량 통계
- 총 충전시간, 총 충전량

In [289]:
ch_time_day_by_local = df2.groupby(['지역'])['충전시간'].sum()
ch_time_day_by_local

지역
강원도       19394 days 08:15:45
경기도       44943 days 22:18:50
경상남도      19393 days 17:46:57
경상북도      32622 days 19:07:20
광주광역시      5026 days 00:52:54
대구광역시      8859 days 12:04:57
대전광역시      7934 days 13:38:40
부산광역시      6641 days 06:56:50
서울특별시     20686 days 09:17:13
세종특별자치시    1547 days 10:32:37
울산광역시      5049 days 12:17:23
인천광역시     11199 days 13:48:21
전라남도      15529 days 21:19:46
전라북도      16610 days 18:03:36
제주특별자치도   16986 days 01:29:37
충청남도      21601 days 11:59:59
충청북도      18996 days 07:16:06
Name: 충전시간, dtype: timedelta64[ns]

In [290]:
ch_time_cal_day_by_local = ch_time_day_by_local.to_frame()
ch_time_cal_day_by_local

,충전시간
지역,
강원도,19394 days 08:15:45
경기도,44943 days 22:18:50
경상남도,19393 days 17:46:57
경상북도,32622 days 19:07:20
광주광역시,5026 days 00:52:54
대구광역시,8859 days 12:04:57
대전광역시,7934 days 13:38:40
부산광역시,6641 days 06:56:50
서울특별시,20686 days 09:17:13


In [291]:
temp1 = ch_time_cal_day_by_local['충전시간'].dt.seconds.sum()
temp1

745611

In [292]:
temp2 = ch_time_cal_day_by_local['충전시간'].dt.days.sum()
temp2

273015

In [293]:
type(temp1)

numpy.int64

In [294]:
ch_time_cal_day_by_local['충전시간'] = ch_time_cal_day_by_local['충전시간'].astype(str)
ch_time_cal_day_by_local = ch_time_cal_day_by_local['충전시간'].str.split(expand=True)
ch_time_cal_day_by_local

,0,1,2
지역,,,
강원도,19394,days,08:15:45
경기도,44943,days,22:18:50
경상남도,19393,days,17:46:57
경상북도,32622,days,19:07:20
광주광역시,5026,days,00:52:54
대구광역시,8859,days,12:04:57
대전광역시,7934,days,13:38:40
부산광역시,6641,days,06:56:50
서울특별시,20686,days,09:17:13


In [295]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9100, 15, 207, 6, 51)

In [296]:
days += hours // 24
hours = hours % 24
days, hours

(23, 15)

In [297]:
ch_time_cal_day_by_local = ch_time_cal_day_by_local.drop([1], axis=1)
ch_time_cal_day_by_local

,0,2
지역,,
강원도,19394,08:15:45
경기도,44943,22:18:50
경상남도,19393,17:46:57
경상북도,32622,19:07:20
광주광역시,5026,00:52:54
대구광역시,8859,12:04:57
대전광역시,7934,13:38:40
부산광역시,6641,06:56:50
서울특별시,20686,09:17:13


In [298]:
ch_time_cal_day_by_local.columns = ['days', 'time']
ch_time_cal_day_by_local

,days,time
지역,,
강원도,19394,08:15:45
경기도,44943,22:18:50
경상남도,19393,17:46:57
경상북도,32622,19:07:20
광주광역시,5026,00:52:54
대구광역시,8859,12:04:57
대전광역시,7934,13:38:40
부산광역시,6641,06:56:50
서울특별시,20686,09:17:13


In [299]:
ch_time_cal_day_by_local['days'] = ch_time_cal_day_by_local['days'].astype(int)

In [300]:
ch_time_cal_day_by_local['months'] = ch_time_cal_day_by_local['days'] // 30
ch_time_cal_day_by_local['days'] = ch_time_cal_day_by_local['days'] % 30
ch_time_cal_day_by_local

,days,time,months
지역,,,
강원도,14,08:15:45,646
경기도,3,22:18:50,1498
경상남도,13,17:46:57,646
경상북도,12,19:07:20,1087
광주광역시,16,00:52:54,167
대구광역시,9,12:04:57,295
대전광역시,14,13:38:40,264
부산광역시,11,06:56:50,221
서울특별시,16,09:17:13,689


In [301]:
ch_time_cal_day_by_local = ch_time_cal_day_by_local[['months', 'days', 'time']]
ch_time_cal_day_by_local

,months,days,time
지역,,,
강원도,646,14,08:15:45
경기도,1498,3,22:18:50
경상남도,646,13,17:46:57
경상북도,1087,12,19:07:20
광주광역시,167,16,00:52:54
대구광역시,295,9,12:04:57
대전광역시,264,14,13:38:40
부산광역시,221,11,06:56:50
서울특별시,689,16,09:17:13


In [302]:
ch_time_cal_day_by_local.shape

(17, 3)

In [303]:
ch_time_cal_day_by_local.columns

Index(['months', 'days', 'time'], dtype='object')

In [304]:
temp = pd.DataFrame({'months':[months], 'days':[days], 'time':[f'{hours}:{minutes}:{seconds}']})
temp

,months,days,time
0,9100,23,15:6:51


In [305]:
ch_time_cal_day_by_local = pd.concat([ch_time_cal_day_by_local, temp])
ch_time_cal_day_by_local

,months,days,time
강원도,646,14,08:15:45
경기도,1498,3,22:18:50
경상남도,646,13,17:46:57
경상북도,1087,12,19:07:20
광주광역시,167,16,00:52:54
대구광역시,295,9,12:04:57
대전광역시,264,14,13:38:40
부산광역시,221,11,06:56:50
서울특별시,689,16,09:17:13
세종특별자치시,51,17,10:32:37


In [311]:
ch_time_cal_day_by_local.columns = [['지역별 총 충전시간']*3, ['months', 'days', 'time']]
ch_time_cal_day_by_local

지역별 총 충전시간               
            months days      time
강원도            646   14  08:15:45
경기도           1498    3  22:18:50
경상남도           646   13  17:46:57
경상북도          1087   12  19:07:20
광주광역시          167   16  00:52:54
대구광역시          295    9  12:04:57
대전광역시          264   14  13:38:40
부산광역시          221   11  06:56:50
서울특별시          689   16  09:17:13
세종특별자치시         51   17  10:32:37
울산광역시          168    9  12:17:23
인천광역시          373    9  13:48:21
전라남도           517   19  21:19:46
전라북도           553   20  18:03:36
제주특별자치도        566    6  01:29:37
충청남도           720    1  11:59:59
충청북도           633    6  07:16:06
0             9100   23   15:6:51

In [312]:
ch_time_cal_day_by_local.to_excel('2020-22년_급속충전_지역별_총_충전시간.xlsx', encoding='utf-8')

#### 총 충전시간(계산)

In [327]:
col = '충전시간(계산)'
ch_time_by_local = df2.groupby(['지역'])[col].sum()
ch_time_by_local

지역
강원도       20379 days 13:42:02
경기도       46362 days 07:47:30
경상남도      20375 days 03:50:30
경상북도      34057 days 10:27:05
광주광역시      5125 days 22:19:49
대구광역시      9433 days 15:16:10
대전광역시      8068 days 03:05:12
부산광역시      6848 days 11:00:24
서울특별시     21362 days 05:17:37
세종특별자치시    1607 days 02:52:01
울산광역시      5269 days 06:06:50
인천광역시     11007 days 01:44:22
전라남도      16178 days 02:20:27
전라북도      16372 days 14:46:27
제주특별자치도   17992 days 11:34:50
충청남도      22417 days 11:17:12
충청북도      19226 days 04:59:21
Name: 충전시간(계산), dtype: timedelta64[ns]

In [328]:
temp = ch_time_by_local.to_frame()
temp

,충전시간(계산)
지역,
강원도,20379 days 13:42:02
경기도,46362 days 07:47:30
경상남도,20375 days 03:50:30
경상북도,34057 days 10:27:05
광주광역시,5125 days 22:19:49
대구광역시,9433 days 15:16:10
대전광역시,8068 days 03:05:12
부산광역시,6848 days 11:00:24
서울특별시,21362 days 05:17:37


In [329]:
temp1 = temp[col].dt.seconds.sum()
temp1

534469

In [330]:
temp2 = temp[col].dt.days.sum()
temp2

282077

In [331]:
temp[col] = temp[col].astype(str)
temp = temp[col].str.split(expand=True)
temp

,0,1,2
지역,,,
강원도,20379,days,13:42:02
경기도,46362,days,07:47:30
경상남도,20375,days,03:50:30
경상북도,34057,days,10:27:05
광주광역시,5125,days,22:19:49
대구광역시,9433,days,15:16:10
대전광역시,8068,days,03:05:12
부산광역시,6848,days,11:00:24
서울특별시,21362,days,05:17:37


In [332]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9402, 17, 148, 27, 49)

In [333]:
days += hours // 24
hours = hours % 24
days, hours

(23, 4)

In [334]:
temp = temp.drop([1], axis=1)
temp

,0,2
지역,,
강원도,20379,13:42:02
경기도,46362,07:47:30
경상남도,20375,03:50:30
경상북도,34057,10:27:05
광주광역시,5125,22:19:49
대구광역시,9433,15:16:10
대전광역시,8068,03:05:12
부산광역시,6848,11:00:24
서울특별시,21362,05:17:37


In [335]:
temp.columns = ['days', 'time']
temp

,days,time
지역,,
강원도,20379,13:42:02
경기도,46362,07:47:30
경상남도,20375,03:50:30
경상북도,34057,10:27:05
광주광역시,5125,22:19:49
대구광역시,9433,15:16:10
대전광역시,8068,03:05:12
부산광역시,6848,11:00:24
서울특별시,21362,05:17:37


In [336]:
temp['days'] = temp['days'].astype(int)

In [337]:
temp['months'] = temp['days'] // 30
temp['days'] = temp['days'] % 30
temp

,days,time,months
지역,,,
강원도,9,13:42:02,679
경기도,12,07:47:30,1545
경상남도,5,03:50:30,679
경상북도,7,10:27:05,1135
광주광역시,25,22:19:49,170
대구광역시,13,15:16:10,314
대전광역시,28,03:05:12,268
부산광역시,8,11:00:24,228
서울특별시,2,05:17:37,712


In [338]:
temp = temp[['months', 'days', 'time']]
temp

,months,days,time
지역,,,
강원도,679,9,13:42:02
경기도,1545,12,07:47:30
경상남도,679,5,03:50:30
경상북도,1135,7,10:27:05
광주광역시,170,25,22:19:49
대구광역시,314,13,15:16:10
대전광역시,268,28,03:05:12
부산광역시,228,8,11:00:24
서울특별시,712,2,05:17:37


In [339]:
temp.columns

Index(['months', 'days', 'time'], dtype='object')

In [340]:
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[f'{hours}:{minutes}:{seconds}']})
total

,months,days,time
0,9402,23,4:27:49


In [341]:
temp = pd.concat([temp, total])
temp

,months,days,time
강원도,679,9,13:42:02
경기도,1545,12,07:47:30
경상남도,679,5,03:50:30
경상북도,1135,7,10:27:05
광주광역시,170,25,22:19:49
대구광역시,314,13,15:16:10
대전광역시,268,28,03:05:12
부산광역시,228,8,11:00:24
서울특별시,712,2,05:17:37
세종특별자치시,53,17,02:52:01


In [342]:
temp.columns = [['지역별 총 충전시간(계산)']*3, ['months', 'days', 'time']]
temp

지역별 총 충전시간(계산)               
                months days      time
강원도                679    9  13:42:02
경기도               1545   12  07:47:30
경상남도               679    5  03:50:30
경상북도              1135    7  10:27:05
광주광역시              170   25  22:19:49
대구광역시              314   13  15:16:10
대전광역시              268   28  03:05:12
부산광역시              228    8  11:00:24
서울특별시              712    2  05:17:37
세종특별자치시             53   17  02:52:01
울산광역시              175   19  06:06:50
인천광역시              366   27  01:44:22
전라남도               539    8  02:20:27
전라북도               545   22  14:46:27
제주특별자치도            599   22  11:34:50
충청남도               747    7  11:17:12
충청북도               640   26  04:59:21
0                 9402   23   4:27:49

In [343]:
temp.to_excel('2020-22년_급속충전_지역별_총_충전시간(계산).xlsx', encoding='utf-8')

#### 충전량

In [147]:
ch_vol_by_local = df2.groupby(['지역'])['충전량'].sum()
ch_vol_by_local

지역
강원도        16232572.48
경기도        37043650.51
경상남도       17777425.02
경상북도       27188951.50
광주광역시       3935268.34
대구광역시       7842277.45
대전광역시       6176372.79
부산광역시       5405663.34
서울특별시      16046052.85
세종특별자치시     1298904.64
울산광역시       4329300.12
인천광역시       8821079.64
전라남도       12601306.22
전라북도       14003630.09
제주특별자치도    12207723.80
충청남도       18808718.07
충청북도       15520247.58
Name: 충전량, dtype: float64

In [33]:
ch_vol_by_local.to_excel('2020-22년_급속충전_지역별_총_충전량.xlsx', encoding='utf-8')

### 년도별 충전시간, 충전량 통계
- 총 충전시간, 충전량

In [71]:
ch_time_day_by_year = df2.groupby(['충전시작_년도'])['충전시간'].sum()
ch_time_day_by_year

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [72]:
ch_time_day_by_year.to_excel('2020-22년_급속충전_년도별_총_충전시간.xlsx', encoding='utf-8')

### 년도, 월별 충전시간, 충전량 통계
- 총 충전시간, 총 충전량

#### 총 충전시간

In [568]:
col = '충전시간'
ch_cal_time_by_ym = df2.groupby(['충전시작_년도', '충전시작_월'])[col].sum()
ch_cal_time_by_ym.head()

충전시작_년도  충전시작_월
2020     1        4844 days 06:04:49
         2        3807 days 14:53:40
         3        4220 days 18:42:13
         4        5200 days 19:02:23
         5        6698 days 16:05:39
Name: 충전시간, dtype: timedelta64[ns]

In [569]:
ch_cal_time_by_ym = ch_cal_time_by_ym.to_frame()
ch_cal_time_by_ym.head()

충전시간
충전시작_년도 충전시작_월                   
2020    1      4844 days 06:04:49
        2      3807 days 14:53:40
        3      4220 days 18:42:13
        4      5200 days 19:02:23
        5      6698 days 16:05:39

In [570]:
temp1 = ch_cal_time_by_ym[col].dt.seconds.sum()
temp1

1264011

In [571]:
temp2 = ch_cal_time_by_ym[col].dt.days.sum()
temp2

273009

In [572]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9100, 9, 351, 6, 51)

In [573]:
days += hours // 24
hours = hours % 24
days, hours

(23, 15)

In [574]:
ch_cal_time_by_ym[col] = ch_cal_time_by_ym[col].astype(str)
temp = ch_cal_time_by_ym[col].str.split(expand=True)
temp

0     1         2
충전시작_년도 충전시작_월                       
2020    1        4844  days  06:04:49
        2        3807  days  14:53:40
        3        4220  days  18:42:13
        4        5200  days  19:02:23
        5        6698  days  16:05:39
        6        8425  days  07:42:45
        7        9948  days  17:47:42
        8        9994  days  13:12:00
        9        7887  days  09:02:08
        10       9809  days  02:56:50
        11       9686  days  05:46:34
        12       8772  days  14:40:03
2021    1        6700  days  00:31:35
        2        5972  days  01:10:01
        3        6318  days  03:25:10
        4        5981  days  18:18:29
        5        5934  days  01:30:44
        6        5927  days  10:26:42
        7        6999  days  08:35:22
        8        7217  days  06:09:09
        9        7222  days  08:48:45
        10       8611  days  17:43:58
        11      10017  days  21:16:40
        12      11864  days  13:04:45
2022    1       11766  days  19:03:00
        2        9909  days  20:56:47
        3       10833  days  12:08:33
        4       11210  days  01:08:59
        5       12025  days  11:31:03
        6       12032  days  13:22:44
        7       13605  days  05:53:35
        8       13577  days  10:04:04

In [575]:
temp = temp.drop([1], axis=1)
temp.columns = ['days', 'time']
temp['months'] = temp['days'].astype(int) // 30
temp['days'] = temp['days'].astype(int) % 30
temp

days      time  months
충전시작_년도 충전시작_월                        
2020    1         14  06:04:49     161
        2         27  14:53:40     126
        3         20  18:42:13     140
        4         10  19:02:23     173
        5          8  16:05:39     223
        6         25  07:42:45     280
        7         18  17:47:42     331
        8          4  13:12:00     333
        9         27  09:02:08     262
        10        29  02:56:50     326
        11        26  05:46:34     322
        12        12  14:40:03     292
2021    1         10  00:31:35     223
        2          2  01:10:01     199
        3         18  03:25:10     210
        4         11  18:18:29     199
        5         24  01:30:44     197
        6         17  10:26:42     197
        7          9  08:35:22     233
        8         17  06:09:09     240
        9         22  08:48:45     240
        10         1  17:43:58     287
        11        27  21:16:40     333
        12        14  13:04:45     395
2022    1          6  19:03:00     392
        2          9  20:56:47     330
        3          3  12:08:33     361
        4         20  01:08:59     373
        5         25  11:31:03     400
        6          2  13:22:44     401
        7         15  05:53:35     453
        8         17  10:04:04     452

In [576]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [577]:
sp1 = temp.iloc[:12, :]
sp1

days      time  months
충전시작_년도 충전시작_월                        
2020    1         14  06:04:49     161
        2         27  14:53:40     126
        3         20  18:42:13     140
        4         10  19:02:23     173
        5          8  16:05:39     223
        6         25  07:42:45     280
        7         18  17:47:42     331
        8          4  13:12:00     333
        9         27  09:02:08     262
        10        29  02:56:50     326
        11        26  05:46:34     322
        12        12  14:40:03     292

In [578]:
sp2 = temp.iloc[12:24, :]
sp2

days      time  months
충전시작_년도 충전시작_월                        
2021    1         10  00:31:35     223
        2          2  01:10:01     199
        3         18  03:25:10     210
        4         11  18:18:29     199
        5         24  01:30:44     197
        6         17  10:26:42     197
        7          9  08:35:22     233
        8         17  06:09:09     240
        9         22  08:48:45     240
        10         1  17:43:58     287
        11        27  21:16:40     333
        12        14  13:04:45     395

In [579]:
sp3 = temp.iloc[24:, :]
sp3

days      time  months
충전시작_년도 충전시작_월                        
2022    1          6  19:03:00     392
        2          9  20:56:47     330
        3          3  12:08:33     361
        4         20  01:08:59     373
        5         25  11:31:03     400
        6          2  13:22:44     401
        7         15  05:53:35     453
        8         17  10:04:04     452

In [580]:
ch_cal_time_by_y = df2.groupby(['충전시작_년도'])[col].sum()
ch_cal_time_by_y.head()

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [581]:
ch_cal_time_by_y = ch_cal_time_by_y.to_frame()
ch_cal_time_by_y[col] = ch_cal_time_by_y[col].astype(str)
ch_cal_time_by_y

,충전시간
충전시작_년도,
2020,89296 days 01:56:46
2021,88766 days 15:01:20
2022,94960 days 22:08:44.999999488


In [582]:
ch_cal_time_by_y = ch_cal_time_by_y[col].str.split(expand=True)
ch_cal_time_by_y

,0,1,2
충전시작_년도,,,
2020,89296,days,01:56:46
2021,88766,days,15:01:20
2022,94960,days,22:08:44.999999488


In [583]:
ch_cal_time_by_y = ch_cal_time_by_y.drop([1], axis=1)
ch_cal_time_by_y

,0,2
충전시작_년도,,
2020,89296,01:56:46
2021,88766,15:01:20
2022,94960,22:08:44.999999488


In [584]:
ch_cal_time_by_y[0] = ch_cal_time_by_y[0].astype(int)
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      int32 
 1   2       3 non-null      object
dtypes: int32(1), object(1)
memory usage: 60.0+ bytes


In [585]:
temp20 = ch_cal_time_by_y.iloc[0]
temp21 = ch_cal_time_by_y.iloc[1]
temp22 = ch_cal_time_by_y.iloc[2]

In [586]:
d20, t20 = temp20[0], temp20[2]
months20 = d20 // 30
days20 = d20 % 30
months20, days20, t20

(2976, 16, '01:56:46')

In [587]:
d21, t21 = temp21[0], temp21[2]
months21 = d21 // 30
days21 = d21 % 30
months21, days21, t21

(2958, 26, '15:01:20')

In [588]:
d22,t22 = temp22[0], temp22[2]
months22 = d22 // 30
days22 = d22 % 30
months22, days22, t22

(3165, 10, '22:08:44.999999488')

In [589]:
sp1.columns

Index(['days', 'time', 'months'], dtype='object')

In [590]:
total20 = pd.DataFrame({'days':[days20], 'time':[t20], 'months':[months20]})
total20

,days,time,months
0,16,01:56:46,2976


In [591]:
total21 = pd.DataFrame({'days':[days21], 'time':[t21], 'months':[months21]})
total21

,days,time,months
0,26,15:01:20,2958


In [592]:
total22 = pd.DataFrame({'days':[days22], 'time':[t22], 'months':[months22]})
total22

,days,time,months
0,10,22:08:44.999999488,3165


In [593]:
temp20 = pd.concat([sp1, total20])
temp20

,days,time,months
"(2020, 1)",14,06:04:49,161
"(2020, 2)",27,14:53:40,126
"(2020, 3)",20,18:42:13,140
"(2020, 4)",10,19:02:23,173
"(2020, 5)",8,16:05:39,223
"(2020, 6)",25,07:42:45,280
"(2020, 7)",18,17:47:42,331
"(2020, 8)",4,13:12:00,333
"(2020, 9)",27,09:02:08,262
"(2020, 10)",29,02:56:50,326


In [594]:
temp21 = pd.concat([sp2, total21])
temp21

,days,time,months
"(2021, 1)",10,00:31:35,223
"(2021, 2)",2,01:10:01,199
"(2021, 3)",18,03:25:10,210
"(2021, 4)",11,18:18:29,199
"(2021, 5)",24,01:30:44,197
"(2021, 6)",17,10:26:42,197
"(2021, 7)",9,08:35:22,233
"(2021, 8)",17,06:09:09,240
"(2021, 9)",22,08:48:45,240
"(2021, 10)",1,17:43:58,287


In [595]:
temp22 = pd.concat([sp3, total22])
temp22

,days,time,months
"(2022, 1)",6,19:03:00,392
"(2022, 2)",9,20:56:47,330
"(2022, 3)",3,12:08:33,361
"(2022, 4)",20,01:08:59,373
"(2022, 5)",25,11:31:03,400
"(2022, 6)",2,13:22:44,401
"(2022, 7)",15,05:53:35,453
"(2022, 8)",17,10:04:04,452
0,10,22:08:44.999999488,3165


In [596]:
temp01 = pd.concat([temp20, temp21])
temp02 = pd.concat([temp01, temp22])
temp02

,days,time,months
"(2020, 1)",14,06:04:49,161
"(2020, 2)",27,14:53:40,126
"(2020, 3)",20,18:42:13,140
"(2020, 4)",10,19:02:23,173
"(2020, 5)",8,16:05:39,223
"(2020, 6)",25,07:42:45,280
"(2020, 7)",18,17:47:42,331
"(2020, 8)",4,13:12:00,333
"(2020, 9)",27,09:02:08,262
"(2020, 10)",29,02:56:50,326


In [597]:
temp = temp02.copy()
temp

,days,time,months
"(2020, 1)",14,06:04:49,161
"(2020, 2)",27,14:53:40,126
"(2020, 3)",20,18:42:13,140
"(2020, 4)",10,19:02:23,173
"(2020, 5)",8,16:05:39,223
"(2020, 6)",25,07:42:45,280
"(2020, 7)",18,17:47:42,331
"(2020, 8)",4,13:12:00,333
"(2020, 9)",27,09:02:08,262
"(2020, 10)",29,02:56:50,326


In [598]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [599]:
temp = temp[['months', 'days', 'time']]
temp

,months,days,time
"(2020, 1)",161,14,06:04:49
"(2020, 2)",126,27,14:53:40
"(2020, 3)",140,20,18:42:13
"(2020, 4)",173,10,19:02:23
"(2020, 5)",223,8,16:05:39
"(2020, 6)",280,25,07:42:45
"(2020, 7)",331,18,17:47:42
"(2020, 8)",333,4,13:12:00
"(2020, 9)",262,27,09:02:08
"(2020, 10)",326,29,02:56:50


In [600]:
time = f'{hours}:{minutes}:{seconds}'
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[time]})
total

,months,days,time
0,9100,23,15:6:51


In [601]:
temp = pd.concat([temp, total])
temp

,months,days,time
"(2020, 1)",161,14,06:04:49
"(2020, 2)",126,27,14:53:40
"(2020, 3)",140,20,18:42:13
"(2020, 4)",173,10,19:02:23
"(2020, 5)",223,8,16:05:39
"(2020, 6)",280,25,07:42:45
"(2020, 7)",331,18,17:47:42
"(2020, 8)",333,4,13:12:00
"(2020, 9)",262,27,09:02:08
"(2020, 10)",326,29,02:56:50


In [602]:
temp.to_excel('2020-22년_급속충전_년월별_총_충전시간.xlsx', encoding=True)

#### 년, 월별 총 충전시간

In [403]:
col = '충전시간'
ch_cal_time_by_ym = df2.groupby(['충전시작_년도', '충전시작_월'])[col].sum()
ch_cal_time_by_ym

충전시작_년도  충전시작_월
2020     1         4844 days 06:04:49
         2         3807 days 14:53:40
         3         4220 days 18:42:13
         4         5200 days 19:02:23
         5         6698 days 16:05:39
         6         8425 days 07:42:45
         7         9948 days 17:47:42
         8         9994 days 13:12:00
         9         7887 days 09:02:08
         10        9809 days 02:56:50
         11        9686 days 05:46:34
         12        8772 days 14:40:03
2021     1         6700 days 00:31:35
         2         5972 days 01:10:01
         3         6318 days 03:25:10
         4         5981 days 18:18:29
         5         5934 days 01:30:44
         6         5927 days 10:26:42
         7         6999 days 08:35:22
         8         7217 days 06:09:09
         9         7222 days 08:48:45
         10        8611 days 17:43:58
         11       10017 days 21:16:40
         12       11864 days 13:04:45
2022     1        11766 days 19:03:00
         2         9909 days 20:56

In [427]:
col = '충전시간'
ch_cal_time_by_y = df2.groupby(['충전시작_년도'])[col].sum()
ch_cal_time_by_y

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [428]:
ch_cal_time_by_y = ch_cal_time_by_y.to_frame()
ch_cal_time_by_y

,충전시간
충전시작_년도,
2020,89296 days 01:56:46
2021,88766 days 15:01:20
2022,94960 days 22:08:44.999999488


In [434]:
ch_cal_time_by_y['충전시간'] = ch_cal_time_by_y['충전시간'].astype(str)
ch_cal_time_by_y

,충전시간
충전시작_년도,
2020,89296 days 01:56:46
2021,88766 days 15:01:20
2022,94960 days 22:08:44.999999488


In [435]:
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   충전시간    3 non-null      object
dtypes: object(1)
memory usage: 48.0+ bytes


In [438]:
ch_cal_time_by_y = ch_cal_time_by_y['충전시간'].str.split(expand=True)
ch_cal_time_by_y

,0,1,2
충전시작_년도,,,
2020,89296,days,01:56:46
2021,88766,days,15:01:20
2022,94960,days,22:08:44.999999488


In [440]:
ch_cal_time_by_y = ch_cal_time_by_y.drop([1], axis=1)
ch_cal_time_by_y

,0,2
충전시작_년도,,
2020,89296,01:56:46
2021,88766,15:01:20
2022,94960,22:08:44.999999488


In [443]:
ch_cal_time_by_y[0] = ch_cal_time_by_y[0].astype(int)
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      int32 
 1   2       3 non-null      object
dtypes: int32(1), object(1)
memory usage: 60.0+ bytes


In [444]:
temp2020 = ch_cal_time_by_y.iloc[0]
temp2021 = ch_cal_time_by_y.iloc[1]
temp2022 = ch_cal_time_by_y.iloc[2]

In [448]:
d = temp2022[0]
months = d // 30
days = d % 30
months, days

(3165, 10)

In [404]:
ch_cal_time_by_ym = ch_cal_time_by_ym.to_frame()
ch_cal_time_by_ym

충전시간
충전시작_년도 충전시작_월                    
2020    1       4844 days 06:04:49
        2       3807 days 14:53:40
        3       4220 days 18:42:13
        4       5200 days 19:02:23
        5       6698 days 16:05:39
        6       8425 days 07:42:45
        7       9948 days 17:47:42
        8       9994 days 13:12:00
        9       7887 days 09:02:08
        10      9809 days 02:56:50
        11      9686 days 05:46:34
        12      8772 days 14:40:03
2021    1       6700 days 00:31:35
        2       5972 days 01:10:01
        3       6318 days 03:25:10
        4       5981 days 18:18:29
        5       5934 days 01:30:44
        6       5927 days 10:26:42
        7       6999 days 08:35:22
        8       7217 days 06:09:09
        9       7222 days 08:48:45
        10      8611 days 17:43:58
        11     10017 days 21:16:40
        12     11864 days 13:04:45
2022    1      11766 days 19:03:00
        2       9909 days 20:56:47
        3      10833 days 12:08:33
        4      11210 days 01:08:59
        5      12025 days 11:31:03
        6      12032 days 13:22:44
        7      13605 days 05:53:35
        8      13577 days 10:04:04

In [405]:
temp1 = ch_cal_time_by_ym[col].dt.seconds.sum()
temp1

1264011

In [406]:
temp2 = ch_cal_time_by_ym[col].dt.days.sum()
temp2

273009

In [407]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9100, 9, 351, 6, 51)

In [408]:
days += hours // 24
hours = hours % 24
days, hours

(23, 15)

In [409]:
ch_cal_time_by_ym[col] = ch_cal_time_by_ym[col].astype(str)
ch_cal_time_by_ym

충전시간
충전시작_년도 충전시작_월                     
2020    1        4844 days 06:04:49
        2        3807 days 14:53:40
        3        4220 days 18:42:13
        4        5200 days 19:02:23
        5        6698 days 16:05:39
        6        8425 days 07:42:45
        7        9948 days 17:47:42
        8        9994 days 13:12:00
        9        7887 days 09:02:08
        10       9809 days 02:56:50
        11       9686 days 05:46:34
        12       8772 days 14:40:03
2021    1        6700 days 00:31:35
        2        5972 days 01:10:01
        3        6318 days 03:25:10
        4        5981 days 18:18:29
        5        5934 days 01:30:44
        6        5927 days 10:26:42
        7        6999 days 08:35:22
        8        7217 days 06:09:09
        9        7222 days 08:48:45
        10       8611 days 17:43:58
        11      10017 days 21:16:40
        12      11864 days 13:04:45
2022    1       11766 days 19:03:00
        2        9909 days 20:56:47
        3       10833 days 12:08:33
        4       11210 days 01:08:59
        5       12025 days 11:31:03
        6       12032 days 13:22:44
        7       13605 days 05:53:35
        8       13577 days 10:04:04

In [410]:
temp = ch_cal_time_by_ym[col].str.split(expand=True)
temp

0     1         2
충전시작_년도 충전시작_월                       
2020    1        4844  days  06:04:49
        2        3807  days  14:53:40
        3        4220  days  18:42:13
        4        5200  days  19:02:23
        5        6698  days  16:05:39
        6        8425  days  07:42:45
        7        9948  days  17:47:42
        8        9994  days  13:12:00
        9        7887  days  09:02:08
        10       9809  days  02:56:50
        11       9686  days  05:46:34
        12       8772  days  14:40:03
2021    1        6700  days  00:31:35
        2        5972  days  01:10:01
        3        6318  days  03:25:10
        4        5981  days  18:18:29
        5        5934  days  01:30:44
        6        5927  days  10:26:42
        7        6999  days  08:35:22
        8        7217  days  06:09:09
        9        7222  days  08:48:45
        10       8611  days  17:43:58
        11      10017  days  21:16:40
        12      11864  days  13:04:45
2022    1       11766  days  19:03:00
        2        9909  days  20:56:47
        3       10833  days  12:08:33
        4       11210  days  01:08:59
        5       12025  days  11:31:03
        6       12032  days  13:22:44
        7       13605  days  05:53:35
        8       13577  days  10:04:04

In [411]:
temp = temp.drop([1], axis=1)
temp

0         2
충전시작_년도 충전시작_월                 
2020    1        4844  06:04:49
        2        3807  14:53:40
        3        4220  18:42:13
        4        5200  19:02:23
        5        6698  16:05:39
        6        8425  07:42:45
        7        9948  17:47:42
        8        9994  13:12:00
        9        7887  09:02:08
        10       9809  02:56:50
        11       9686  05:46:34
        12       8772  14:40:03
2021    1        6700  00:31:35
        2        5972  01:10:01
        3        6318  03:25:10
        4        5981  18:18:29
        5        5934  01:30:44
        6        5927  10:26:42
        7        6999  08:35:22
        8        7217  06:09:09
        9        7222  08:48:45
        10       8611  17:43:58
        11      10017  21:16:40
        12      11864  13:04:45
2022    1       11766  19:03:00
        2        9909  20:56:47
        3       10833  12:08:33
        4       11210  01:08:59
        5       12025  11:31:03
        6       12032  13:22:44
        7       13605  05:53:35
        8       13577  10:04:04

In [412]:
temp.columns = ['days', 'time']
temp

days      time
충전시작_년도 충전시작_월                 
2020    1        4844  06:04:49
        2        3807  14:53:40
        3        4220  18:42:13
        4        5200  19:02:23
        5        6698  16:05:39
        6        8425  07:42:45
        7        9948  17:47:42
        8        9994  13:12:00
        9        7887  09:02:08
        10       9809  02:56:50
        11       9686  05:46:34
        12       8772  14:40:03
2021    1        6700  00:31:35
        2        5972  01:10:01
        3        6318  03:25:10
        4        5981  18:18:29
        5        5934  01:30:44
        6        5927  10:26:42
        7        6999  08:35:22
        8        7217  06:09:09
        9        7222  08:48:45
        10       8611  17:43:58
        11      10017  21:16:40
        12      11864  13:04:45
2022    1       11766  19:03:00
        2        9909  20:56:47
        3       10833  12:08:33
        4       11210  01:08:59
        5       12025  11:31:03
        6       12032  13:22:44
        7       13605  05:53:35
        8       13577  10:04:04

In [413]:
temp['months'] = temp['days'].astype(int) // 30
temp['days'] = temp['days'].astype(int) % 30
temp

days      time  months
충전시작_년도 충전시작_월                        
2020    1         14  06:04:49     161
        2         27  14:53:40     126
        3         20  18:42:13     140
        4         10  19:02:23     173
        5          8  16:05:39     223
        6         25  07:42:45     280
        7         18  17:47:42     331
        8          4  13:12:00     333
        9         27  09:02:08     262
        10        29  02:56:50     326
        11        26  05:46:34     322
        12        12  14:40:03     292
2021    1         10  00:31:35     223
        2          2  01:10:01     199
        3         18  03:25:10     210
        4         11  18:18:29     199
        5         24  01:30:44     197
        6         17  10:26:42     197
        7          9  08:35:22     233
        8         17  06:09:09     240
        9         22  08:48:45     240
        10         1  17:43:58     287
        11        27  21:16:40     333
        12        14  13:04:45     395
2022    1          6  19:03:00     392
        2          9  20:56:47     330
        3          3  12:08:33     361
        4         20  01:08:59     373
        5         25  11:31:03     400
        6          2  13:22:44     401
        7         15  05:53:35     453
        8         17  10:04:04     452

In [414]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [415]:
temp = temp[['months', 'days', 'time']]
temp

months  days      time
충전시작_년도 충전시작_월                        
2020    1          161    14  06:04:49
        2          126    27  14:53:40
        3          140    20  18:42:13
        4          173    10  19:02:23
        5          223     8  16:05:39
        6          280    25  07:42:45
        7          331    18  17:47:42
        8          333     4  13:12:00
        9          262    27  09:02:08
        10         326    29  02:56:50
        11         322    26  05:46:34
        12         292    12  14:40:03
2021    1          223    10  00:31:35
        2          199     2  01:10:01
        3          210    18  03:25:10
        4          199    11  18:18:29
        5          197    24  01:30:44
        6          197    17  10:26:42
        7          233     9  08:35:22
        8          240    17  06:09:09
        9          240    22  08:48:45
        10         287     1  17:43:58
        11         333    27  21:16:40
        12         395    14  13:04:45
2022    1          392     6  19:03:00
        2          330     9  20:56:47
        3          361     3  12:08:33
        4          373    20  01:08:59
        5          400    25  11:31:03
        6          401     2  13:22:44
        7          453    15  05:53:35
        8          452    17  10:04:04

In [416]:
temp.columns

Index(['months', 'days', 'time'], dtype='object')

In [417]:
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[f'{hours}:{minutes}:{seconds}']})
total

,months,days,time
0,9100,23,15:6:51


In [418]:
temp.to_excel('2020-22년_급속충전_년월별_총_충전시간.xlsx', encoding='utf-8')

#### 총 충전시간(계산)

In [603]:
col = '충전시간(계산)'
ch_cal_time_by_ym = df2.groupby(['충전시작_년도', '충전시작_월'])[col].sum()
ch_cal_time_by_ym.head()

충전시작_년도  충전시작_월
2020     1        5111 days 07:36:54
         2        4030 days 10:04:17
         3        4128 days 11:36:27
         4        4900 days 20:11:06
         5        6183 days 22:26:24
Name: 충전시간(계산), dtype: timedelta64[ns]

In [604]:
ch_cal_time_by_ym = ch_cal_time_by_ym.to_frame()
ch_cal_time_by_ym.head()

충전시간(계산)
충전시작_년도 충전시작_월                   
2020    1      5111 days 07:36:54
        2      4030 days 10:04:17
        3      4128 days 11:36:27
        4      4900 days 20:11:06
        5      6183 days 22:26:24

In [605]:
temp1 = ch_cal_time_by_ym[col].dt.seconds.sum()
temp1

1398469

In [606]:
temp2 = ch_cal_time_by_ym[col].dt.days.sum()
temp2

282067

In [607]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9402, 7, 388, 27, 49)

In [608]:
days += hours // 24
hours = hours % 24
days, hours

(23, 4)

In [609]:
ch_cal_time_by_ym[col] = ch_cal_time_by_ym[col].astype(str)
temp = ch_cal_time_by_ym[col].str.split(expand=True)

In [610]:
temp = temp.drop([1], axis=1)
temp.columns = ['days', 'time']
temp['months'] = temp['days'].astype(int) // 30
temp['days'] = temp['days'].astype(int) % 30

In [611]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [612]:
sp1 = temp.iloc[:12, :]
sp1

days      time  months
충전시작_년도 충전시작_월                        
2020    1         11  07:36:54     170
        2         10  10:04:17     134
        3         18  11:36:27     137
        4         10  20:11:06     163
        5          3  22:26:24     206
        6         11  18:02:20     251
        7          3  12:15:12     292
        8         29  04:45:22     296
        9          6  07:09:15     237
        10        12  04:42:48     301
        11        23  08:37:36     316
        12        28  17:01:32     293

In [613]:
sp2 = temp.iloc[12:24, :]
sp2

days      time  months
충전시작_년도 충전시작_월                        
2021    1         10  17:06:06     232
        2          8  23:23:36     209
        3         27  16:40:25     219
        4         27  11:25:41     208
        5         19  08:51:42     207
        6         23  16:48:38     207
        7          0  13:25:41     245
        8         21  11:25:24     264
        9         16  00:14:44     268
        10        22  22:42:28     319
        11        14  09:48:00     366
        12         3  17:25:22     430

In [614]:
sp3 = temp.iloc[24:, :]
sp3

days      time  months
충전시작_년도 충전시작_월                        
2022    1         16  06:53:00     425
        2         20  18:15:50     358
        3          0  02:18:10     394
        4          7  02:13:02     406
        5         16  15:54:21     437
        6         26  21:10:27     435
        7         12  02:56:44     483
        8          6  04:59:15     479

In [615]:
ch_cal_time_by_y = df2.groupby(['충전시작_년도'])[col].sum()
ch_cal_time_by_y.head()

충전시작_년도
2020    84050 days 00:29:12.999999488
2021    95417 days 01:17:47.000000512
2022   102616 days 02:40:48.999999488
Name: 충전시간(계산), dtype: timedelta64[ns]

In [616]:
ch_cal_time_by_y = ch_cal_time_by_y.to_frame()
ch_cal_time_by_y[col] = ch_cal_time_by_y[col].astype(str)
ch_cal_time_by_y

,충전시간(계산)
충전시작_년도,
2020,84050 days 00:29:12.999999488
2021,95417 days 01:17:47.000000512
2022,102616 days 02:40:48.999999488


In [617]:
ch_cal_time_by_y = ch_cal_time_by_y[col].str.split(expand=True)
ch_cal_time_by_y

,0,1,2
충전시작_년도,,,
2020,84050,days,00:29:12.999999488
2021,95417,days,01:17:47.000000512
2022,102616,days,02:40:48.999999488


In [618]:
ch_cal_time_by_y = ch_cal_time_by_y.drop([1], axis=1)
ch_cal_time_by_y

,0,2
충전시작_년도,,
2020,84050,00:29:12.999999488
2021,95417,01:17:47.000000512
2022,102616,02:40:48.999999488


In [619]:
ch_cal_time_by_y[0] = ch_cal_time_by_y[0].astype(int)
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      int32 
 1   2       3 non-null      object
dtypes: int32(1), object(1)
memory usage: 60.0+ bytes


In [620]:
temp20 = ch_cal_time_by_y.iloc[0]
temp21 = ch_cal_time_by_y.iloc[1]
temp22 = ch_cal_time_by_y.iloc[2]

In [621]:
d20, t20 = temp20[0], temp20[2]
months20 = d20 // 30
days20 = d20 % 30
months20, days20, t20

(2801, 20, '00:29:12.999999488')

In [622]:
d21, t21 = temp21[0], temp21[2]
months21 = d21 // 30
days21 = d21 % 30
months21, days21, t21

(3180, 17, '01:17:47.000000512')

In [623]:
d22,t22 = temp22[0], temp22[2]
months22 = d22 // 30
days22 = d22 % 30
months22, days22, t22

(3420, 16, '02:40:48.999999488')

In [624]:
sp1.columns

Index(['days', 'time', 'months'], dtype='object')

In [625]:
total20 = pd.DataFrame({'days':[days20], 'time':[t20], 'months':[months20]})
total20

,days,time,months
0,20,00:29:12.999999488,2801


In [626]:
total21 = pd.DataFrame({'days':[days21], 'time':[t21], 'months':[months21]})
total21

,days,time,months
0,17,01:17:47.000000512,3180


In [627]:
total22 = pd.DataFrame({'days':[days22], 'time':[t22], 'months':[months22]})
total22

,days,time,months
0,16,02:40:48.999999488,3420


In [628]:
temp20 = pd.concat([sp1, total20])
temp20

,days,time,months
"(2020, 1)",11,07:36:54,170
"(2020, 2)",10,10:04:17,134
"(2020, 3)",18,11:36:27,137
"(2020, 4)",10,20:11:06,163
"(2020, 5)",3,22:26:24,206
"(2020, 6)",11,18:02:20,251
"(2020, 7)",3,12:15:12,292
"(2020, 8)",29,04:45:22,296
"(2020, 9)",6,07:09:15,237
"(2020, 10)",12,04:42:48,301


In [629]:
temp21 = pd.concat([sp2, total21])
temp21

,days,time,months
"(2021, 1)",10,17:06:06,232
"(2021, 2)",8,23:23:36,209
"(2021, 3)",27,16:40:25,219
"(2021, 4)",27,11:25:41,208
"(2021, 5)",19,08:51:42,207
"(2021, 6)",23,16:48:38,207
"(2021, 7)",0,13:25:41,245
"(2021, 8)",21,11:25:24,264
"(2021, 9)",16,00:14:44,268
"(2021, 10)",22,22:42:28,319


In [630]:
temp22 = pd.concat([sp3, total22])
temp22

,days,time,months
"(2022, 1)",16,06:53:00,425
"(2022, 2)",20,18:15:50,358
"(2022, 3)",0,02:18:10,394
"(2022, 4)",7,02:13:02,406
"(2022, 5)",16,15:54:21,437
"(2022, 6)",26,21:10:27,435
"(2022, 7)",12,02:56:44,483
"(2022, 8)",6,04:59:15,479
0,16,02:40:48.999999488,3420


In [631]:
temp01 = pd.concat([temp20, temp21])
temp02 = pd.concat([temp01, temp22])
temp02

,days,time,months
"(2020, 1)",11,07:36:54,170
"(2020, 2)",10,10:04:17,134
"(2020, 3)",18,11:36:27,137
"(2020, 4)",10,20:11:06,163
"(2020, 5)",3,22:26:24,206
"(2020, 6)",11,18:02:20,251
"(2020, 7)",3,12:15:12,292
"(2020, 8)",29,04:45:22,296
"(2020, 9)",6,07:09:15,237
"(2020, 10)",12,04:42:48,301


In [632]:
temp = temp02.copy()
temp

,days,time,months
"(2020, 1)",11,07:36:54,170
"(2020, 2)",10,10:04:17,134
"(2020, 3)",18,11:36:27,137
"(2020, 4)",10,20:11:06,163
"(2020, 5)",3,22:26:24,206
"(2020, 6)",11,18:02:20,251
"(2020, 7)",3,12:15:12,292
"(2020, 8)",29,04:45:22,296
"(2020, 9)",6,07:09:15,237
"(2020, 10)",12,04:42:48,301


In [633]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [634]:
temp = temp[['months', 'days', 'time']]

In [635]:
time = f'{hours}:{minutes}:{seconds}'
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[time]})
total

,months,days,time
0,9402,23,4:27:49


In [636]:
temp = pd.concat([temp, total])
temp

,months,days,time
"(2020, 1)",170,11,07:36:54
"(2020, 2)",134,10,10:04:17
"(2020, 3)",137,18,11:36:27
"(2020, 4)",163,10,20:11:06
"(2020, 5)",206,3,22:26:24
"(2020, 6)",251,11,18:02:20
"(2020, 7)",292,3,12:15:12
"(2020, 8)",296,29,04:45:22
"(2020, 9)",237,6,07:09:15
"(2020, 10)",301,12,04:42:48


In [637]:
temp.to_excel('2020-22년_급속충전_년월별_총_충전시간(계산).xlsx', encoding=True)

## 주소 분리
- 샘플수
    - 신주소 : 7,749,439개
    - 구주소 : 3,864,010개
- 한 주소에 2개 다른 주소가 입력
    - 동일 지역
        - 신주소 : 20,837개
        - 구주소 : 99,756개
    - 다른 지역
        - 신주소 : 0개
        - 구주소 : 5,427개
    - raw데이터의 입력값 지역이 있는 주소로 병합
        - 예시) 전라남도 무안군 청계면 영산로 1854-16 전라남도 무안군 청계면 영산로 1854-16 -> 전라남도 무안군 청계면 영산로 1854-16
        - 예시) 충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272 -> 충청남도 청양군 청양읍 군량리 533
        - 예시) 전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2 -> 전라북도 순창군 풍산면 반월리 12-4
- 도로명 수정
    - "도로명주소 안내" 참고하여 지역 코드와 맞추기위해
        - 예시) 첨단 벤처로 108번길 -> 첨단벤처로108번길

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
dtypes: float64(1), int64(15), object(11)
memory usage: 2.4+ GB


In [220]:
ad = df2['주소'].copy()

In [221]:
temp = ad.to_list()
len(temp)

11613449

In [222]:
old_p = re.compile('[가-힣0-9]+동 |[가-힣0-9]+동^[가-힣]|[가-힣0-9]+동[0-9]+|[가-힣0-9]+동$|[가-힣0-9]+리 |[가-힣0-9]+리[0-9]+|[가-힣0-9]+리^[가-힣]|[가-힣0-9]+리$')
s = '제주특별자치도 제주시 외도1동 482-1'
print(old_p.search(s))

<re.Match object; span=(12, 17), match='외도1동 '>


In [223]:
s = '경기도 시흥시 조남동'
print(old_p.search(s))

<re.Match object; span=(8, 11), match='조남동'>


In [224]:
s = '제주특별자치도 제주시 외도1동482-1'
print(old_p.search(s))

<re.Match object; span=(12, 19), match='외도1동482'>


In [225]:
s = '강원도 평창군 용평면 탑거리길 63 , 내고장 으뜸상품 판매장 앞'
print(old_p.search(s))

None


In [226]:
s = '전라북도 남원시 산내면 덕동리 406'
print(old_p.search(s))

<re.Match object; span=(13, 17), match='덕동리 '>


In [227]:
s = '전라북도 남원시 산내면 덕동리406'
print(old_p.search(s))

<re.Match object; span=(13, 19), match='덕동리406'>


In [228]:
s = '충청북도 보은군 마로면 적암리'
print(old_p.search(s))

<re.Match object; span=(13, 16), match='적암리'>


In [230]:
# 오류 주소
    # 신 : 충청북도 단양군 적성면 기동리
    # 신 : 제주특별자치도 제주시 외도1동 482-1
    # 구 : 강원도 평창군 용평면 탑거리길 63 , 내고장 으뜸상품 판매장 앞
    # 신 : 경기도 시흥시 조남동
    # 신 : 7802945
old_adr = []
old_idx = []
new_adr = []
new_idx = []
# new_p = re.compile('[가-힣]+로{1}|[가-힣]+로[0-9]+|[가-힣]+길|[가-힣0-9]+번길')
old_p = re.compile('[가-힣0-9]+동 |[가-힣0-9]+동^[가-힣]|[가-힣0-9]+동[0-9]+|[가-힣0-9]+동$|[가-힣0-9]+리 |[가-힣0-9]+리[0-9]+|[가-힣0-9]+리^[가-힣]|[가-힣0-9]+리$')
for i, s in enumerate(temp):
    # if new_p.search(s):
    if old_p.search(s):
        old_adr.append(s)
        old_idx.append(i)
    else:
        new_adr.append(s)
        new_idx.append(i)

print(len(new_idx))
print(len(old_idx))
print(len(temp))
print(len(new_idx) + len(old_idx) == len(temp))

7749439
3864010
11613449
True


In [231]:
start_idx = 10*0
new_adr[start_idx:start_idx+10]

['광주광역시 북구 첨단 벤처로 108번길 9',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 노고단로1068',
 '전라남도 무안군 망운면 공항로 970-260',
 '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)',
 '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)',
 '전라북도 전주시 완산구 신촌3길24번지',
 '전라북도 전주시 완산구 신촌3길24번지',
 '전라북도 전주시 완산구 신촌3길24번지']

In [232]:
for i in new_adr:
    if '구로구' in i:
        print(i)
        break

서울특별시 구로구 경인로 661  지하2층 L


In [233]:
# 빈칸 2개 있는것 1개로 변환
for i, s in enumerate(new_adr):
    if '  ' in s:
        new_adr[i] = s.replace('  ', ' ')

In [234]:
for i in new_adr:
    if '구로구' in i:
        print(i)
        break

서울특별시 구로구 경인로 661 지하2층 L


In [235]:
for i in new_adr:
    if '구로구 개봉동' in i:
        print(i)
        break

In [236]:
start_idx = 100
new_adr[start_idx:start_idx + 10]

['제주특별자치도 제주시 오일장서길 26',
 '제주특별자치도 제주시 오일장서길 26',
 '제주특별자치도 제주시 오일장서길 26',
 '제주특별자치도 제주시 오일장서길 26',
 '제주특별자치도 제주시 오일장서길 26',
 '제주특별자치도 제주시 중앙로25길 17',
 '제주특별자치도 제주시 중앙로25길 17',
 '제주특별자치도 제주시 조천읍 중산간동로 601',
 '제주특별자치도 제주시 조천읍 중산간동로 601',
 '제주특별자치도 제주시 조천읍 중산간동로 601']

In [237]:
start_idx = 100
old_adr[start_idx:start_idx + 10]

['경상남도 김해시 대동면 월촌리 770',
 '경상남도 김해시 대동면 월촌리 770',
 '경상남도 김해시 대동면 월촌리 770',
 '경상남도 김해시 대동면 월촌리 770',
 '경상북도 성주군 초전면 용봉리 129-5',
 '경상북도 성주군 초전면 용봉리 129-5',
 '강원도 인제군 기린면 현리 634-7',
 '강원도 정선군 남면 무릉리 784-4',
 '강원도 정선군 남면 무릉리 784-4',
 '강원도 정선군 남면 무릉리 784-4']

### 신주소, 구주소 데이터 프레임 지정

In [810]:
new_adr_df = df2.iloc[new_idx]
old_adr_df = df2.iloc[old_idx]

In [811]:
new_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7749439 entries, 4 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
dtypes: float64(1), int64(15), object(11)
memory usage: 1.6+ GB


In [812]:
old_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3864010 entries, 0 to 11665820
Data columns (total 27 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
dtypes: float64(1), int64(15), object(11)
memory usage: 825.4+ MB


### 신주소 정리
- 7749439 개

In [813]:
temp = new_adr_df['주소'].to_list()
temp[:10]

['광주광역시 북구 첨단 벤처로 108번길 9',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 노고단로1068',
 '전라남도 무안군 망운면 공항로 970-260',
 '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)',
 '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)',
 '전라북도 전주시 완산구 신촌3길24번지',
 '전라북도 전주시 완산구 신촌3길24번지',
 '전라북도 전주시 완산구 신촌3길24번지']

In [814]:
len(temp)

7749439

#### 주소 표기 수정
- "전국 도로명 부여사유 조회(2022.08.31기준)" 파일 참고

##### 컴마(,) 제거

In [815]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

세종특별자치시 연기면 정안세종로 1527 은하수공원, 화장시설 주차장


In [816]:
for i, s in enumerate(temp):
    if ',' in s:
        temp[i] = s.replace(',', '')

In [817]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

##### 도로명 형식에 맞게 수정

In [818]:
temp[:5]

['광주광역시 북구 첨단 벤처로 108번길 9',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 노고단로1068',
 '전라남도 무안군 망운면 공항로 970-260']

In [819]:
'광주광역시 북구 첨단 벤처로 108번길 9'.replace('첨단 벤처로', '첨단벤처로')

'광주광역시 북구 첨단벤처로 108번길 9'

In [820]:
for i, one in enumerate(temp):
    if '첨단 벤처로' in one:
        temp[i] = one.replace('첨단 벤처로', '첨단벤처로')

In [821]:
temp[:10]

['광주광역시 북구 첨단벤처로 108번길 9',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 노고단로1068',
 '전라남도 무안군 망운면 공항로 970-260',
 '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)',
 '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)',
 '전라북도 전주시 완산구 신촌3길24번지',
 '전라북도 전주시 완산구 신촌3길24번지',
 '전라북도 전주시 완산구 신촌3길24번지']

##### -길과 숫자 띄어쓰기

In [822]:
n_p = re.compile('([가-힣0-9]+길)([0-9]+)')
s = '전라북도 전주시 완산구 신촌3길24번지'
print(n_p.sub('\g<1> \g<2>', s))

전라북도 전주시 완산구 신촌3길 24번지


In [823]:
s = '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)'
print(n_p.sub('\g<1> \g<2>', s))

전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)


In [824]:
# -길과 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+길)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', s)

In [825]:
start_idx = 10*11
temp[start_idx:start_idx+10]

['강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '경기도 이천시 마장면 중부고속도로82',
 '경기도 여주시 가남읍 여주남로 722 (본두리)']

##### -로와 숫자 띄어쓰기

In [826]:
n_p = re.compile('([가-힣0-9]+로)([0-9]+)')
s = '경기도 이천시 마장면 중부고속도로82'
print(n_p.sub('\g<1> \g<2>', s))

경기도 이천시 마장면 중부고속도로 82


In [827]:
s = '강원도 강릉시 성산면 영동고속도로 232 (보광리)'
print(n_p.sub('\g<1> \g<2>', s))

강원도 강릉시 성산면 영동고속도로 232 (보광리)


In [828]:
# -로와 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+로)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', s)

In [829]:
start_idx = 10*11
temp[start_idx:start_idx+10]

['강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 231 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '경기도 여주시 가남읍 여주남로 722 (본두리)']

In [830]:
start_idx = 10*1000
temp[start_idx:start_idx+10]

['경상북도 상주시 화서면 터골길 51-25',
 '강원도 강릉시 성산면 영동고속도로 232 (보광리)',
 '경상북도 안동시 풍산읍 풍산태사로 1903-27',
 '경상북도 안동시 풍산읍 풍산태사로 1903-27',
 '경상북도 안동시 풍산읍 풍산태사로 1903-27',
 '경상북도 안동시 풍산읍 풍산태사로 1913-40',
 '경상북도 안동시 풍산읍 풍산태사로 1913-40',
 '전라남도 순천시 남승룡로 66  국가정원1호길 47 서문 주차장 우측',
 '전라남도 고흥군 과역면 무궁화길 12',
 '전라남도 고흥군 과역면 무궁화길 12']

##### -로-번-길 붙이기

In [831]:
n_p = re.compile('([가-힣0-9]+로) ([0-9]+번[가-힣]*길)')
s = '광주광역시 북구 첨단벤처로 108번길 9'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(9, 20), match='첨단벤처로 108번길'>
광주광역시 북구 첨단벤처로108번길 9


In [832]:
# -로-번-길 붙이기
n_p = re.compile('([가-힣0-9]+로) ([0-9]+번[가-힣]*길)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', s)

In [833]:
for one in temp:
    if '공항로' in one:
        print(one)
        break

전라남도 무안군 망운면 공항로 970-260


In [834]:
for one in temp:
    if '앞산순환로' in one:
        print(one)
        break

대구광역시 남구 앞산순환로 574-110


In [835]:
n_p = re.compile('([가-힣0-9]+로) ([0-9]+[가-힣]*길)')
s = '강남구 양재대로 31길'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(4, 12), match='양재대로 31길'>
강남구 양재대로31길


In [836]:
s = '강남구 양재대로'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
강남구 양재대로


In [837]:
s = '강남구 양재대로31길'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
강남구 양재대로31길


In [838]:
# -로-길 붙이기
n_p = re.compile('([가-힣0-9]+로) ([0-9]+[가-힣]*길)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', s)

##### -앙-길 붙이기

In [839]:
n_p = re.compile('([가-힣0-9]+앙) ([0-9]+[가-힣]*길)')
s = '대구광역시 달성군 창리중앙 3길'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(10, 17), match='창리중앙 3길'>
대구광역시 달성군 창리중앙3길


In [840]:
s = '대구광역시 달성군 창리중앙3길'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
대구광역시 달성군 창리중앙3길


In [841]:
s = '대구광역시 달성군 창리중앙'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
대구광역시 달성군 창리중앙


In [842]:
# -앙-길 붙이기
n_p = re.compile('([가-힣0-9]+앙) ([0-9]+[가-힣]*길)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', s)

##### -장-길 붙이기

In [843]:
n_p = re.compile('([가-힣0-9]+장) ([0-9]+[가-힣]*길)')
s = '대구광역시 남구 관문시장 1길'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(9, 16), match='관문시장 1길'>
대구광역시 남구 관문시장1길


In [844]:
s = '대구광역시 남구 관문시장1길'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
대구광역시 남구 관문시장1길


In [845]:
s = '대구광역시 남구 관문시장'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
대구광역시 남구 관문시장


In [846]:
# -장-길 붙이기
n_p = re.compile('([가-힣0-9]+장) ([0-9]+[가-힣]*길)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', s)

In [847]:
for one in temp:
    if '대경' in one:
        print(one)
        break

In [848]:
for one in temp:
    if '빛그린' in one:
        print(one)
        break

In [849]:
for one in temp:
    if '빛동' in one:
        print(one)
        break

In [850]:
for one in temp:
    if '둥지' in one:
        print(one)
        break

#### 복수 주소 오류
- 한 주소에 2개 주소 들어있는 것 처리

##### 동일지역
- index(new_adr_df) : 1446642

In [851]:
new_adr_df.loc[1446642, '주소']

'전라남도 무안군 청계면 영산로 1854-16 전라남도 무안군 청계면 영산로 1854-16'

In [852]:
'전라남도 무안군 망운면 운해로 1436-10'.replace(' 전라남도', '$전라남도').split('$')

['전라남도 무안군 망운면 운해로 1436-10']

In [853]:
temp.index('전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9')

850930

In [854]:
temp[850930]

'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'

In [855]:
temp_si = new_adr_df['시군구'].to_list()
temp_si[:5]

['북구', '구례군', '구례군', '구례군', '무안군']

In [856]:
count = 0
for i, one in enumerate(temp):
    si = list(one.split())[0]
    one = one.replace(' ' + si, '$' + si)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
    for j, two in enumerate(multi_adr):
        if temp_si[i] in two:
            temp[i] = multi_adr[j]

In [857]:
temp[850930]

'전라남도 무안군 망운면 운해로 1436-10'

In [858]:
count

20837

##### 다른지역
- index(new_adr_df) : 

In [859]:
new_adr_df['지역'].unique()

array(['광주광역시', '전라남도', '전라북도', '제주특별자치도', '세종특별자치시', '서울특별시', '대전광역시',
       '경기도', '인천광역시', '경상남도', '강원도', '충청북도', '경상북도', '충청남도', '울산광역시',
       '대구광역시', '부산광역시'], dtype=object)

In [879]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [959]:
re.split(sep, '광주광역시 북구')

['광주광역시 북구']

In [960]:
len(re.split(sep, '광주광역시 북구'))

1

In [957]:
re.split(sep, '광주광역시 충청북도')

['광주광역시', '']

In [958]:
len(re.split(sep, '광주광역시 충청북도'))

2

In [882]:
count = 0
for i, one in enumerate(temp):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        count += 1

In [883]:
count

0

In [886]:
new_adr_df['주소'] = temp

C:\Users\kbj\AppData\Local\Temp\ipykernel_15028\2793901220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_adr_df['주소'] = temp


In [887]:
new_adr_df['주소'].head()

4            광주광역시 북구 첨단벤처로108번길 9
8            전라남도 구례군 산동면 산업로 2319
9            전라남도 구례군 산동면 산업로 2319
10    강원도 강릉시 성산면 영동고속도로 232 (보광리)
15        전라남도 무안군 망운면 공항로 970-260
Name: 주소, dtype: object

In [888]:
# data : 7749439 개
new_adr_df['주소'].info()

<class 'pandas.core.series.Series'>
Int64Index: 7749439 entries, 4 to 11665821
Series name: 주소
Non-Null Count    Dtype 
--------------    ----- 
7749439 non-null  object
dtypes: object(1)
memory usage: 376.2+ MB


### 구주소 정리
- 3810504 개

In [889]:
temp = old_adr_df['주소'].to_list()
temp[:10]

['강원도 인제군 상남면 하남리 339-7(양양방향)',
 '강원도 인제군 상남면 하남리 339-7(양양방향)',
 '강원도 인제군 상남면 하남리 339-7(양양방향)',
 '강원도 인제군 상남면 하남리 339-7(양양방향)',
 '전라남도 순천시 해룡면 남가리 733-73',
 '전라남도 순천시 해룡면 남가리 733-73',
 '전라남도 순천시 해룡면 남가리 733-73',
 '전라남도 장성군 북하면 약수리 252-6',
 '전라남도 장성군 북하면 신성리 427-2',
 '전라북도 부안군 변산면 격포리 283-1']

##### 컴마(,) 제거

In [890]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

전라남도 화순군 화순읍 훈리 31, 군청 후면 주차장


In [891]:
for i, s in enumerate(temp):
    if ',' in s:
        temp[i] = s.replace(',', '')

In [892]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

##### -동과 숫자 띄어쓰기

In [893]:
n_p = re.compile('([가-힣0-9]+동)([0-9]+)')
s = '제주특별자치도 제주시 연동2334-4'
print(n_p.sub('\g<1> \g<2>', s))

제주특별자치도 제주시 연동 2334-4


In [894]:
s = '전라북도 정읍시 내장동 52'
print(n_p.sub('\g<1> \g<2>', s))

전라북도 정읍시 내장동 52


In [895]:
# -동과 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+동)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', s)

In [896]:
start_idx = 10*1
temp[start_idx:start_idx+10]

['전라남도 영암군 영암읍 개신리 484-47',
 '전라북도 남원시 산내면 덕동리 406',
 '전라북도 정읍시 내장동 52',
 '전라북도 정읍시 내장동 169-1',
 '전라북도 무주군 설천면 삼공리 411',
 '전라북도 무주군 설천면 삼공리 411',
 '전라북도 무주군 설천면 삼공리 411',
 '전라북도 정읍시 내장동 52',
 '전라북도 정읍시 내장동 52',
 '전라북도 정읍시 내장동 52']

##### -리과 숫자 띄어쓰기

In [897]:
n_p = re.compile('([가-힣0-9]+리)([0-9]+)')
s = '전라남도 영암군 영암읍 개신리484-47'
print(n_p.sub('\g<1> \g<2>', s))

전라남도 영암군 영암읍 개신리 484-47


In [898]:
s = '충청북도 단양군 적성면 기동리'
print(n_p.sub('\g<1> \g<2>', s))

충청북도 단양군 적성면 기동리


In [899]:
s = '충청북도 괴산군 괴산읍 서부리 654'
print(n_p.sub('\g<1> \g<2>', s))

충청북도 괴산군 괴산읍 서부리 654


In [900]:
for i, one in enumerate(temp):
    if n_p.search(one):
        print(one)
        break

전라북도 익산시 함열읍 와리164


In [901]:
# -리과 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+리)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', s)

In [902]:
for i, one in enumerate(temp):
    if n_p.search(one):
        print(one)
        break

#### 복수 주소 정리
- 한 주소에 2개 들어있는 것 처리

##### 동일 지역

In [903]:
old_adr_df['주소'].info()

<class 'pandas.core.series.Series'>
Int64Index: 3864010 entries, 0 to 11665820
Series name: 주소
Non-Null Count    Dtype 
--------------    ----- 
3864010 non-null  object
dtypes: object(1)
memory usage: 59.0+ MB


In [904]:
for i, s in enumerate(old_adr_df['주소']):
    if ('청양군' in s) and ('태안군' in s):
        print(i)
        break

382258


In [905]:
old_adr_df['주소'].to_list()[382258]

'충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272'

In [906]:
temp.index('충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272')

382258

In [907]:
temp[382258]

'충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272'

In [908]:
temp_si = old_adr_df['시군구'].to_list()
temp_si[:5]

['인제군', '인제군', '인제군', '인제군', '순천시']

In [909]:
count = 0
for i, one in enumerate(temp):
    si = list(one.split())[0]
    one = one.replace(' ' + si, '$' + si)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
    for j, two in enumerate(multi_adr):
        if temp_si[i] in two:
            temp[i] = multi_adr[j]

In [910]:
temp[382258]

'충청남도 청양군 청양읍 군량리 533'

In [911]:
count

99756

##### 다른 지역

In [912]:
rep_dict = {" 광주광역시":"$광주광역시" ," 전라남도":"$전라남도"," 전라북도":"$전라북도"," 제주특별자치도":"$제주특별자치도"," 세종특별자치시":"$세종특별자치시"," 서울특별시":"$서울특별시"," 대전광역시":"$대전광역시"," 경기도":"$경기도"," 인천광역시":"$인천광역시"," 경상남도":"$경상남도"," 강원도":"$강원도"," 충청북도":"$충청북도"," 경상북도":"$경상북도"," 충청남도":"$충청남도"," 울산광역시":"$울산광역시"," 대구광역시":"$대구광역시"," 부산광역시":"$부산광역시"}

In [913]:
for k, v in rep_dict.items():
    print(k, v)

 광주광역시 $광주광역시
 전라남도 $전라남도
 전라북도 $전라북도
 제주특별자치도 $제주특별자치도
 세종특별자치시 $세종특별자치시
 서울특별시 $서울특별시
 대전광역시 $대전광역시
 경기도 $경기도
 인천광역시 $인천광역시
 경상남도 $경상남도
 강원도 $강원도
 충청북도 $충청북도
 경상북도 $경상북도
 충청남도 $충청남도
 울산광역시 $울산광역시
 대구광역시 $대구광역시
 부산광역시 $부산광역시


In [914]:
s = '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'
for k, v in rep_dict.items():
    s = s.replace(k, v)
s

'전라북도 순창군 풍산면 반월리 12-4$충청남도 공주시 탄천면 삼각리 523-2'

In [915]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [918]:
count = 0
multi_adr_old_indx = []
for i, one in enumerate(temp):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        multi_adr_old_indx.append(i)
        count += 1

In [919]:
count

5427

In [920]:
multi_adr_old_indx[:5]

[296880, 339425, 564676, 656400, 662447]

In [921]:
temp[296880]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [922]:
count = 0
for i, one in enumerate(temp):
    for k, v in rep_dict.items():
        one = one.replace(k, v)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
    for j, two in enumerate(multi_adr):
        if temp_si[i] in two:
            temp[i] = multi_adr[j]

In [923]:
count

5427

In [924]:
temp[296880]

'전라북도 순창군 풍산면 반월리 12-4'

In [925]:
temp[339425]

'전라북도 순창군 풍산면 반월리 12-4'

In [926]:
old_adr_df['주소'] = temp

C:\Users\kbj\AppData\Local\Temp\ipykernel_15028\1619837769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_adr_df['주소'] = temp


In [927]:
old_adr_df['주소'].head()

0    강원도 인제군 상남면 하남리 339-7(양양방향)
1    강원도 인제군 상남면 하남리 339-7(양양방향)
2    강원도 인제군 상남면 하남리 339-7(양양방향)
3    강원도 인제군 상남면 하남리 339-7(양양방향)
5        전라남도 순천시 해룡면 남가리 733-73
Name: 주소, dtype: object

In [928]:
# data : 3864010 개
old_adr_df['주소'].info()

<class 'pandas.core.series.Series'>
Int64Index: 3864010 entries, 0 to 11665820
Series name: 주소
Non-Null Count    Dtype 
--------------    ----- 
3864010 non-null  object
dtypes: object(1)
memory usage: 59.0+ MB


##### 신주소, 구주소 분리 파일 출력

In [929]:
new_adr_df.to_csv('2020-22년_급속충전_신주소.csv', encoding='utf-8')

In [930]:
old_adr_df.to_csv('2020-22년_급속충전_구주소.csv', encoding='utf-8')

##### 신주소, 구주소 로드(다시 시작)

In [931]:
# new_adr_df = pd.read_csv('2020-22년_급속충전_신주소.csv', index_col=0)
# old_adr_df = pd.read_csv('2020-22년_급속충전_구주소.csv', index_col=0)

### 신주소 상세 분리

#### 상세 분리

In [932]:
temp = new_adr_df['주소'].str.split(expand=True)
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
4,광주광역시,북구,첨단벤처로108번길,9,None,None,None,None,None,None,None,None,None
8,전라남도,구례군,산동면,산업로,2319,None,None,None,None,None,None,None,None
9,전라남도,구례군,산동면,산업로,2319,None,None,None,None,None,None,None,None
10,강원도,강릉시,성산면,영동고속도로,232,(보광리),None,None,None,None,None,None,None
15,전라남도,무안군,망운면,공항로,970-260,None,None,None,None,None,None,None,None


In [933]:
# 5번컬럼까지 -길, -로, -번길 존재
# 6번부터 없음
check_p = re.compile('로$|길$|번길$')
for one in range(5, 13):
    print(f'{one}', end=' : ')
    uni = temp[temp[one].notnull() == True][one].unique()
    # print(uni)
    count = 0
    for two in uni:
        if check_p.search(two):
            count += 1
            break
    if count != 0:
        print('Yes')
    else:
        print('No')

5 : Yes
6 : No
7 : No
8 : No
9 : No
10 : No
11 : No
12 : No


In [934]:
temp[5].unique()

array([None, '(보광리)', '화장시설', '1F)', '휴게소', '주차장', '농축산물판매장', '휴게소와',
       '알펜시아', '내고장', '민원봉사실', '(본두리)', '(고림동)', '입구', '지하2층', '수원도시공사',
       '수영장', '본관', '매표소', '표선생활체육관', '곶자왈', '서귀포농업기술센터', '모구리야영장',
       '(외부리)', '(보산리)', '(매류리)', '(신제리)', '(오가리)', '(계내리)', '마리나요트클럽',
       '소형차', '앞', '후포문화복지센터', '복지센터', '14', '우측', '옆', '추자면사무소',
       '환경성질환예방관리센터', '애월읍도서관', '지원사무소', '(가정리)', '(박산리)', '(비월리)',
       '(은교리)', '(남산리)', '(신양리)', '(양덕리)', '(방내리)', '36', '왼편',
       '행담도휴게소(서울방향)', '행담도휴게소(목포방향)', '지하주차장', '뒤편', '(수진동)', '(율암리)',
       '가평휴게소', '정면', '좌측', '23-37', '제2소형주차장', '2616', '136', '1182',
       '자치경찰대', '05', '24', '01', '후문주차장', '매장', '(성하리)', '(용전리)',
       '(오곡리)', '(가천리)', '(옥산리)', '206', '후문', '28', '매표소앞', '(송계리)',
       '(신전리)', '(방목리)', '(양악리)', '(공진리)', '47', '공영주차장', '알뜰휴게소',
       '화성시청사', '주차', '주유소', '지상주차장', '13', '외부화장실', '차량경정비', '고분군', '4층',
       '옥외주차장', '계단쪽', '좌측편', '(하방리)', '(금창리)', '(소매곡리)', '(제비리)',
       '(망상동)', '(주수리)', '(묵곡리

In [935]:
temp[temp[5] == '온달평강3로'].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
822932,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
825340,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
909454,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
938599,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
954186,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None


In [936]:
temp[temp[5] == '온달평강3로'].shape

(808, 13)

In [937]:
temp = temp.fillna(np.nan)

In [938]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
4,광주광역시,북구,첨단벤처로108번길,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,전라남도,구례군,산동면,산업로,2319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,전라남도,구례군,산동면,산업로,2319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,강원도,강릉시,성산면,영동고속도로,232,(보광리),NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,전라남도,무안군,망운면,공항로,970-260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [939]:
new_adr_df2 = pd.concat([new_adr_df, temp], axis=1)
new_adr_df2.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단벤처로108번길 9,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,0 days 00:31:14,21.92,...,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SK 산동2주유소,1,전라남도,구례군,전라남도 구례군 산동면 산업로 2319,DC차데모+AC3상+DC콤보,20200115135355,20200115141450,0 days 00:20:28,14.05,...,산업로,2319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SK 산동2주유소,1,전라남도,구례군,전라남도 구례군 산동면 산업로 2319,DC차데모+AC3상+DC콤보,20200101145057,20200101151847,0 days 00:27:25,18.95,...,산업로,2319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,지리산국립공원 남부 성삼재 주차장,2,전라남도,구례군,강원도 강릉시 성산면 영동고속도로 232 (보광리),DC차데모+AC3상+DC콤보,20200126112202,20200126113312,0 days 00:10:47,6.16,...,영동고속도로,232,(보광리),NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,무안공항,1,전라남도,무안군,전라남도 무안군 망운면 공항로 970-260,DC차데모+AC3상+DC콤보,20200101090653,20200101094311,0 days 00:35:56,25.44,...,공항로,970-260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 신주소 상세 분리 확인

In [940]:
new_adr_df2[['주소', 0, 1, 2, 3, 4]].tail()

,주소,0,1,2,3,4
11665815,경기도 구리시 체육관로 74,경기도,구리시,체육관로,74,NaN
11665816,서울특별시 성동구 둘레길 47-5,서울특별시,성동구,둘레길,47-5,NaN
11665817,경기도 용인시 수지구 경부고속도로 400 (죽전동),경기도,용인시,수지구,경부고속도로,400
11665818,강원도 원주시 단구로 150,강원도,원주시,단구로,150,NaN
11665821,강원도 강릉시 성산면 영동고속도로 232 (보광리),강원도,강릉시,성산면,영동고속도로,232


In [956]:
new_adr_df2[['주소', 0, 1, 2, 3, 4]].head()

,주소,0,1,2,3,4
4,광주광역시 북구 첨단벤처로108번길 9,광주광역시,북구,첨단벤처로108번길,9,NaN
8,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,산업로,2319
9,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,산업로,2319
10,강원도 강릉시 성산면 영동고속도로 232 (보광리),강원도,강릉시,성산면,영동고속도로,232
15,전라남도 무안군 망운면 공항로 970-260,전라남도,무안군,망운면,공항로,970-260


##### 신주소 상세 분리 파일 출력

In [941]:
new_adr_df2.to_csv('2020-22년_급속충전_신주소_상세분리.csv', encoding='utf-8')

### 구주소 상세분리

#### 상세분리

In [942]:
temp = old_adr_df['주소'].str.split(expand=True)
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None
1,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None
2,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None
3,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None
5,전라남도,순천시,해룡면,남가리,733-73,None,None,None,None,None,None,None


In [943]:
# 5번컬럼까지 -길, -로, -번길 존재
# 6번부터 없음
check_p = re.compile('동$|리$')
for one in range(4, 12):
    print(f'{one}', end=' : ')
    uni = temp[temp[one].notnull() == True][one].unique()
    # print(uni)
    count = 0
    for two in uni:
        if check_p.search(two):
            count += 1
            break
    if count != 0:
        print('Yes')
    else:
        print('No')

4 : Yes
5 : Yes
6 : Yes
7 : Yes
8 : No
9 : Yes
10 : No
11 : No


In [946]:
temp[6].unique()

array([None, '후면', '주차장', '좌측', '우측', '앞', '옆', '384-27)', '뒤', '맞은편',
       '사업단', '건물', '오른쪽', '20구역', '증설주차장', 'LPG충전소', '교동', '입구', '흡연실',
       '왼쪽', '2층', '맨끝쪽', '287기둥옆', '맨끝', '건물입구', '가03', '공영주차장', '산업단지',
       '(나성동', '지하주차장', '근처', '사무실', '430기둥', '끝', '안쪽', '지하4층', '관리사무소',
       '버스주차장', '1282-16)', '마금산온천', '밑', '820-10)', '매표소앞', '정면', '옥상',
       'KTX', '주차구역', '1층', '501)', '2'], dtype=object)

In [947]:
temp[5].unique()

array([None, '군청', '산방산', '무릉문화의집', '좌측', '우측', '630-5', '소형주차장', '입구',
       '(향교동)', '28', '안쪽', '일원', '179-6', '재릉종합관광안내센터', '604-3', '주차타워',
       '뒤편', '보조경기장', '(여양3로', '끝', '온정종합터미널', '주차장', '1층', '새마을금고',
       '스포츠', '고엽제전우회', '별관', '주민센터', '그린빌딩', '북측', '북주차장(우측)', '휴게소와',
       '건너편', '앞', '경주시', '499-1)', '97-1)', '공중화장실', '2층주차장', '입구앞',
       '본관', '필지', '지하', '지상2층', '구청', '4층매장입구옆', '지하1층', '천마산군립공원',
       '사랑방', '제2공영주차장', '대마전기차', '182-131', '북구청', 'B1', '2-15', '마트입구',
       '후생동', '검품장', '753', '정면', '230-1', '44', '인근', '15', '26', '옆',
       '정문', '지상', '번지', '(백운리', '407-1', '골프장', '16-1', '제주현대미술관',
       '(신관리', '403-3', '영암국제카트장', '3층', '24-3', '건물', '485', '공영주차장',
       '370-1', '386-2', '대형', '환승주차장', '천의삼거리', '292-5', '30-2', '57-26',
       '312', '307', '954-4', '679', '7-10', '664-5', '민원봉사실', '42',
       '353', '391-3', '30-1', '376-1', '11', '765-161', '(덕곡리', '577-2',
       '23-1', '35-4', '1127-28', '6', '140-1', '164-3', '7-6', '271-22

In [950]:
temp = temp.fillna(np.nan)

In [951]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,강원도,인제군,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,강원도,인제군,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,강원도,인제군,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강원도,인제군,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,전라남도,순천시,해룡면,남가리,733-73,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [952]:
old_adr_df2 = pd.concat([old_adr_df, temp], axis=1)
old_adr_df2.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,2,3,4,5,6,7,8,9,10,11
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200117193331,20200117201417,0 days 00:40:00,25.47,...,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200130143545,20200130150645,0 days 00:30:31,17.57,...,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200105005728,20200105011329,0 days 00:15:35,6.98,...,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC콤보,20200120120215,20200120124708,0 days 00:41:00,15.30,...,상남면,하남리,339-7(양양방향),NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,현대블루핸즈 지봉점,1,전라남도,순천시,전라남도 순천시 해룡면 남가리 733-73,DC차데모+AC3상+DC콤보,20200112150247,20200112154347,0 days 00:40:00,28.72,...,해룡면,남가리,733-73,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 주소 상세 분리 확인

In [953]:
old_adr_df2[['주소', 0, 1, 2, 3, 4]].tail()

,주소,0,1,2,3,4
11665807,경기도 안산시 상록구 사동 1271-11,경기도,안산시,상록구,사동,1271-11
11665811,경기도 의정부시 호원동 229-160,경기도,의정부시,호원동,229-160,NaN
11665813,서울특별시 서초구 반포동 49-8,서울특별시,서초구,반포동,49-8,NaN
11665819,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,인천광역시,서구,당하동,1097-5,검단4동
11665820,대구광역시 남구 봉덕동 1272-18,대구광역시,남구,봉덕동,1272-18,NaN


In [955]:
old_adr_df2[['주소', 0, 1, 2, 3, 4]].head()

,주소,0,1,2,3,4
0,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,339-7(양양방향)
1,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,339-7(양양방향)
2,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,339-7(양양방향)
3,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,339-7(양양방향)
5,전라남도 순천시 해룡면 남가리 733-73,전라남도,순천시,해룡면,남가리,733-73


##### 구주소 상세 분리 파일 출력

In [954]:
old_adr_df2.to_csv('2020-22년_급속충전_구주소_상세분리.csv', encoding='utf-8')

## 분석

##### 신, 구주소 상세분리 파일 로드(다시 시작)

In [3]:
new_adr_df2 = pd.read_csv('2020-22년_급속충전_신주소_상세분리.csv', index_col=0)
old_adr_df2 = pd.read_csv('2020-22년_급속충전_구주소_상세분리.csv', index_col=0)

C:\Users\kbj\AppData\Local\Temp\ipykernel_1120\2564991263.py:1: DtypeWarning: Columns (11,40) have mixed types. Specify dtype option on import or set low_memory=False.
  new_adr_df2 = pd.read_csv('2020-22년_급속충전_신주소_상세분리.csv', index_col=0)
C:\Users\kbj\AppData\Local\Temp\ipykernel_1120\2564991263.py:2: DtypeWarning: Columns (11,39) have mixed types. Specify dtype option on import or set low_memory=False.
  old_adr_df2 = pd.read_csv('2020-22년_급속충전_구주소_상세분리.csv', index_col=0)


In [4]:
new_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7749439 entries, 4 to 11665821
Data columns (total 40 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
 27  0             object 
 28  1             object 
 29  2             object 
 30  3             obj

In [5]:
old_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3864010 entries, 0 to 11665820
Data columns (total 39 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
 27  0             object 
 28  1             object 
 29  2             object 
 30  3             obj

##### reset index

In [6]:
new_adr_df2 = new_adr_df2.reset_index(drop=True)
old_adr_df2 = old_adr_df2.reset_index(drop=True)

In [7]:
new_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7749439 entries, 0 to 7749438
Data columns (total 40 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
 27  0             object 
 28  1             object 
 29  2             object 
 30  3             obje

In [8]:
old_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3864010 entries, 0 to 3864009
Data columns (total 39 columns):
 #   Column        Dtype  
---  ------        -----  
 0   충전소명          object 
 1   충전기ID         int64  
 2   지역            object 
 3   시군구           object 
 4   주소            object 
 5   충전기타입         object 
 6   충전시작일시        int64  
 7   충전종료일시        int64  
 8   충전시간          object 
 9   충전량           float64
 10  충전용량          object 
 11  충전시작_년도       int64  
 12  충전시작_월        int64  
 13  충전시작_일        int64  
 14  충전시작_시        int64  
 15  충전시작_분        int64  
 16  충전시작_초        int64  
 17  충전시작일시(변환)    object 
 18  충전종료_년도       int64  
 19  충전종료_월        int64  
 20  충전종료_일        int64  
 21  충전종료_시        int64  
 22  충전종료_분        int64  
 23  충전종료_초        int64  
 24  충전종료일시(변환)    object 
 25  충전시간(계산)      object 
 26  raw값과_계산값_차이  object 
 27  0             object 
 28  1             object 
 29  2             object 
 30  3             obje

##### 충전시간 타입 변경

In [18]:
new_adr_df2['충전시간'] = pd.to_timedelta(new_adr_df2['충전시간'])
new_adr_df2['충전시간'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7749439 entries, 0 to 7749438
Series name: 충전시간
Non-Null Count    Dtype          
--------------    -----          
7749439 non-null  timedelta64[ns]
dtypes: timedelta64[ns](1)
memory usage: 59.1 MB


### 교체 충전기 수 예측
- 충전기 별 충전량/충전시간 데이터 활용
- 같은 지역내에 동일한 "충전기ID" 존재

In [20]:
new_adr_df2.columns

Index(['충전소명', '충전기ID', '지역', '시군구', '주소', '충전기타입', '충전시작일시', '충전종료일시', '충전시간',
       '충전량', '충전용량', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분',
       '충전시작_초', '충전시작일시(변환)', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시',
       '충전종료_분', '충전종료_초', '충전종료일시(변환)', '충전시간(계산)', 'raw값과_계산값_차이', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
      dtype='object')

In [21]:
temp = new_adr_df2[['충전기ID', '지역', '시군구', '충전기타입', '충전시간', '충전량']]
temp.head()

,충전기ID,지역,시군구,충전기타입,충전시간,충전량
0,1,광주광역시,북구,DC차데모+AC3상+DC콤보,0 days 00:31:14,21.92
1,1,전라남도,구례군,DC차데모+AC3상+DC콤보,0 days 00:20:28,14.05
2,1,전라남도,구례군,DC차데모+AC3상+DC콤보,0 days 00:27:25,18.95
3,2,전라남도,구례군,DC차데모+AC3상+DC콤보,0 days 00:10:47,6.16
4,1,전라남도,무안군,DC차데모+AC3상+DC콤보,0 days 00:35:56,25.44


In [22]:
temp['지역'].unique()

array(['광주광역시', '전라남도', '전라북도', '제주특별자치도', '세종특별자치시', '서울특별시', '대전광역시',
       '경기도', '인천광역시', '경상남도', '강원도', '충청북도', '경상북도', '충청남도', '울산광역시',
       '대구광역시', '부산광역시'], dtype=object)

In [23]:
temp['충전기ID'].unique()

array([ 1,  2, 11,  3,  4,  5, 12, 52, 14, 18, 51, 10, 13,  6,  8, 15, 17,
       42, 21, 41, 44, 43, 22, 24, 23, 16,  7, 91, 88, 80, 89, 86, 83, 85,
       84, 87, 45, 81, 82, 46, 47, 26, 25, 92, 28, 27, 95, 98, 96, 97, 93,
       94], dtype=int64)

In [24]:
temp[temp['지역'] == '서울특별시']['충전기ID'].unique()

array([ 1,  3,  4,  5,  2, 52, 12, 13, 11,  6, 14, 21, 22,  8,  7, 41, 42,
       44, 45, 43, 24, 23, 26, 25], dtype=int64)

In [28]:
temp[temp['지역'] == '서울특별시']['충전기ID'].value_counts()

1     404482
2      52729
22     42123
21     37271
12     28079
11     26816
41     11340
45      8639
44      8049
3       7917
4       7459
42      6339
43      6216
23      5414
52      5070
24      5002
5       3903
26      3707
6       3336
8       3249
25      2697
7       2221
13       961
14       214
Name: 충전기ID, dtype: int64

In [25]:
temp[temp['지역'] == '경기도']['충전기ID'].unique()

array([12,  1,  2, 11, 52, 51,  5,  3, 14, 13, 22, 21,  4,  8,  7, 41, 42,
       23,  6, 43, 44, 24, 88, 80, 89, 86, 83, 85, 84, 87, 45, 81, 82, 46,
       91, 26, 28, 27, 25, 95, 98, 96, 97, 93, 92, 94], dtype=int64)

In [26]:
seoul = temp[temp['지역'] == '서울특별시']
seoul.head()

,충전기ID,지역,시군구,충전기타입,충전시간,충전량
19,1,서울특별시,영등포구,DC차데모+AC3상+DC콤보,0 days 00:18:50,11.43
20,1,서울특별시,영등포구,DC차데모+AC3상+DC콤보,0 days 00:25:58,6.99
21,3,서울특별시,영등포구,DC콤보,0 days 00:26:40,5.77
22,3,서울특별시,영등포구,DC콤보,0 days 00:41:00,21.72
23,4,서울특별시,영등포구,DC콤보,0 days 00:18:41,8.06


In [29]:
seoul['시군구'].unique()

array(['영등포구', '마포구', '성동구', '강서구', '관악구', '강남구', '강동구', '광진구', '노원구',
       '강북구', '서초구', '종로구', '송파구', '중구', '동대문구', '동작구', '서대문구', '용산구',
       '양천구', '은평구', '성북구', '중랑구', '구로구', '금천구', '도봉구'], dtype=object)

In [30]:
seoul[seoul['시군구'] == '영등포구']['충전기ID'].unique()

array([ 1,  3,  4,  5, 52,  2,  6], dtype=int64)

In [40]:
seoul[seoul['시군구'] == '영등포구']['충전기ID'].value_counts()

1     48225
2     11116
52     5070
5      1853
4      1849
6      1391
3      1379
Name: 충전기ID, dtype: int64

##### 특정 지역 충전기 확인
- '강원도 강릉시 성산면 영동고속도로 231 (보광리)'

In [44]:
new_adr_df2[(new_adr_df2['시군구'] == '영등포구') & (new_adr_df2['충전기ID'] == 1)]['주소']

19         서울특별시 영등포구 영중로 125 지상주차장(외부 1F)
20         서울특별시 영등포구 영중로 125 지상주차장(외부 1F)
173           강원도 강릉시 성산면 영동고속도로 232 (보광리)
1107                           강남구 양재대로31길
1143                    서울특별시 영등포구 영신로 183
                        ...               
7748975                 서울특별시 영등포구 영신로 183
7749022                 서울특별시 영등포구 대림로 244
7749136                        강남구 양재대로31길
7749195                 서울특별시 영등포구 영신로 183
7749384    서울특별시 영등포구 영중로 125 지상주차장(외부 1F)
Name: 주소, Length: 48225, dtype: object

In [49]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 1)]['주소']

110        강원도 강릉시 성산면 영동고속도로 231 (보광리)
114        강원도 강릉시 성산면 영동고속도로 232 (보광리)
176                   강원도 강릉시 종합운동장길 69
177                   강원도 강릉시 종합운동장길 69
1163        강원도 강릉시 구정면 회산로 88-35 (제비리)
                       ...             
7748566    강원도 강릉시 성산면 영동고속도로 231 (보광리)
7748640             강원도 강릉시 연곡면 연주로 188
7748773    강원도 강릉시 옥계면 동해고속도로 320 (주수리)
7748956               강원도 강릉시 남산초교길 121
7749179    강원도 강릉시 성산면 영동고속도로 231 (보광리)
Name: 주소, Length: 30582, dtype: object

In [47]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 1) & (new_adr_df2['주소'] == '강원도 강릉시 성산면 영동고속도로 231 (보광리)')][['충전시간', '충전량']].agg(['sum', 'mean'])

,충전시간,충전량
sum,106 days 01:21:20,95304.530000
mean,0 days 00:27:53.047288661,17.400864


In [60]:
new_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7749439 entries, 0 to 7749438
Data columns (total 40 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int64          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int64          
 12  충전시작_월        int64          
 13  충전시작_일        int64          
 14  충전시작_시        int64          
 15  충전시작_분        int64          
 16  충전시작_초        int64          
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int64          
 19  충전종료_월        int64          
 20  충전종료_일        int64          
 21  충전종료_시        int64          
 22  충전종료_분        int64          
 23  충전종료_초 

In [61]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 1) & (new_adr_df2['주소'] == '강원도 강릉시 성산면 영동고속도로 231 (보광리)')][['충전소명', '충전기ID', '지역', '시군구']]

,충전소명,충전기ID,지역,시군구
110,강릉휴게소(강릉방향),1,강원도,강릉시
1966,강릉휴게소(강릉방향),1,강원도,강릉시
1967,강릉휴게소(강릉방향),1,강원도,강릉시
1968,강릉휴게소(강릉방향),1,강원도,강릉시
3009,강릉휴게소(강릉방향),1,강원도,강릉시
...,...,...,...,...
7740134,강릉휴게소(강릉방향),1,강원도,강릉시
7743069,강릉휴게소(강릉방향),1,강원도,강릉시
7745693,강릉휴게소(강릉방향),1,강원도,강릉시
7748566,강릉휴게소(강릉방향),1,강원도,강릉시


In [50]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 2)]['주소']

111        강원도 강릉시 성산면 영동고속도로 231 (보광리)
112        강원도 강릉시 성산면 영동고속도로 231 (보광리)
113        강원도 강릉시 성산면 영동고속도로 231 (보광리)
115        강원도 강릉시 성산면 영동고속도로 232 (보광리)
1969       강원도 강릉시 성산면 영동고속도로 231 (보광리)
                       ...             
7746628    강원도 강릉시 성산면 영동고속도로 232 (보광리)
7746781             강원도 강릉시 강동면 와천로 463
7747123    강원도 강릉시 성산면 영동고속도로 231 (보광리)
7747384           강원도 강릉시 연곡면 진고개로 2742
7747713    강원도 강릉시 구정면 동해고속도로 345 (제비리)
Name: 주소, Length: 20678, dtype: object

In [51]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 2) & (new_adr_df2['주소'] == '강원도 강릉시 성산면 영동고속도로 231 (보광리)')][['충전시간', '충전량']].agg(['sum', 'mean'])

,충전시간,충전량
sum,66 days 13:39:36,62449.300000
mean,0 days 00:26:23.583700440,17.194191


In [52]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 3)]['주소']

116        강원도 강릉시 성산면 영동고속도로 232 (보광리)
117        강원도 강릉시 성산면 영동고속도로 232 (보광리)
1167       강원도 강릉시 옥계면 동해고속도로 320 (주수리)
1972       강원도 강릉시 성산면 영동고속도로 231 (보광리)
1973       강원도 강릉시 성산면 영동고속도로 231 (보광리)
                       ...             
7745859    강원도 강릉시 옥계면 동해고속도로 320 (주수리)
7746548    강원도 강릉시 옥계면 동해고속도로 320 (주수리)
7747029    강원도 강릉시 성산면 영동고속도로 232 (보광리)
7747161    강원도 강릉시 옥계면 동해고속도로 320 (주수리)
7748078    강원도 강릉시 성산면 영동고속도로 232 (보광리)
Name: 주소, Length: 14657, dtype: object

In [53]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 3) & (new_adr_df2['주소'] == '강원도 강릉시 성산면 영동고속도로 231 (보광리)')][['충전시간', '충전량']].agg(['sum', 'mean'])

,충전시간,충전량
sum,89 days 05:15:44,83053.010000
mean,0 days 00:26:49.299373695,17.338833


In [31]:
seoul[seoul['시군구'] == '마포구']['충전기ID'].unique()

array([ 3,  1,  2,  4, 21, 22, 24, 23, 26, 25], dtype=int64)

In [41]:
seoul[seoul['시군구'] == '마포구']['충전기ID'].value_counts()

1     17867
22    10008
21     9157
23     3422
24     3311
3      3114
4      2595
2      2394
26     1077
25      883
Name: 충전기ID, dtype: int64

In [27]:
temp[temp['지역'] == '서울특별시'].sum()

Timedelta('16691 days 03:18:28')

#### 지역, 시군구, 충전기ID 별 통계

In [32]:
statis = temp.groupby(['지역', '시군구', '충전기ID'])[['충전시간', '충전량']].agg(['sum', 'mean'])
statis

충전시간                                  충전량  \
                             sum                      mean        sum   
지역   시군구 충전기ID                                                          
강원도  강릉시 1     634 days 18:21:12 0 days 00:29:53.331763782  534969.56   
         2     407 days 14:50:52 0 days 00:28:23.174968565  393614.71   
         3     280 days 17:32:14 0 days 00:27:34.849832844  258075.40   
         4      37 days 18:15:31 0 days 00:30:53.710795454   30046.87   
         21      2 days 16:08:10 0 days 00:28:43.059701492    3948.74   
...                          ...                       ...        ...   
충청북도 충주시 44     24 days 07:06:59 0 days 00:31:41.466485507   22005.70   
         45     39 days 02:42:16 0 days 00:33:45.980815347   31799.03   
         81     11 days 19:52:03 0 days 00:25:27.538116591   18351.61   
         82      7 days 05:05:06 0 days 00:25:08.731234866   11535.28   
         91    113 days 03:08:14 0 days 00:29:09.193629205  110505.32   

                           
                     mean  
지역   시군구 충전기ID             
강원도  강릉시 1      17.492955  
         2      19.035434  
         3      17.607655  
         4      17.072085  
         21     29.468209  
...                   ...  
충청북도 충주시 44     19.932699  
         45     19.064167  
         81     27.431405  
         82     27.930460  
         91     19.775469  

[1569 rows x 4 columns]

In [33]:
statis.to_excel('분석/2020-22년_신주소_지역_시군구_충전기ID_충전시간_충전량_통계.xlsx', encoding='utf-8')

### 특정 지역/기간별 충전량

In [54]:
new_adr_df2.columns

Index(['충전소명', '충전기ID', '지역', '시군구', '주소', '충전기타입', '충전시작일시', '충전종료일시', '충전시간',
       '충전량', '충전용량', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분',
       '충전시작_초', '충전시작일시(변환)', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시',
       '충전종료_분', '충전종료_초', '충전종료일시(변환)', '충전시간(계산)', 'raw값과_계산값_차이', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
      dtype='object')

In [55]:
temp = new_adr_df2[['지역', '시군구', '충전시작_년도', '충전시작_월', '충전량']]
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7749439 entries, 0 to 7749438
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   시군구      object 
 2   충전시작_년도  int64  
 3   충전시작_월   int64  
 4   충전량      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 295.6+ MB


In [56]:
statis = temp.groupby(['지역', '시군구', '충전시작_년도', '충전시작_월'])[['충전량']].agg(['sum', 'mean'])
statis

충전량           
                               sum       mean
지역   시군구 충전시작_년도 충전시작_월                      
강원도  강릉시 2020    1        24097.21  15.617116
                 2        19731.80  15.635341
                 3        19599.25  16.170998
                 4        20738.49  17.026675
                 5        23248.08  16.488000
...                            ...        ...
충청북도 충주시 2022    4       108623.48  19.985921
                 5       115388.01  19.891055
                 6       106327.68  20.130193
                 7       139095.34  21.495185
                 8       199168.47  23.559081

[7513 rows x 2 columns]

In [39]:
statis.to_excel('분석/2020-22년_신주소_지역_시군구_년도_월_충전량_통계.xlsx', encoding='utf-8')

# 코드 마지막